# [프로젝트5] 금속분말 생성공정 최적화를 위한 딥러닝 심화


---

## 프로젝트 목표
---
- 모델의 일반화에 대해 이해합니다.
- 일반화를 위한 방법들을 배워봅니다.
- 가중치 규제 방법을 알아봅니다.
- 앙상블 모델을 배워봅니다.
- 정규화를 배워봅니다.
- 데이터 증강 기법을 이해합니다.

## 프로젝트 목차
---
1. **일반화를 위한 방법들:** 일반화를 위한 여러 방법들을 사용해보고 성능을 비교합니다.

2. **가중치 규제:** 가중치 규제 방법을 수행해보고, 성능을 비교해봅니다.

3. **앙상블 모델:** 앙상블 모델을 구현해봅니다.

4. **Dropout:** Dropout을 구현해봅니다.

5. **정규화:** 정규화를 구현해봅니다.

6. **데이터 증강 기법:** 데이터 증강기법을 수행해봅니다.

## 프로젝트 개요
---

이번 프로젝트에서는 다양한 일반화를 위한 방법들을 수행해봅니다.

## 1. 일반화를 위한 방법들
---
금속분말 데이터셋을 이용하여 일반화 방법들을 수행해보겠습니다.


### 1.1 라이브러리 불러오기

In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
import json
import sklearn.metrics

### 1.2 데이터셋 읽어오기

In [2]:
stage1 = {
    'train_X': np.load('./Data/train_data_stage1_X.npy'),
    'train_y': np.load('./Data/train_data_stage1_y.npy'),
    'valid_X': np.load('./Data/valid_data_stage1_X.npy'),
    'valid_y': np.load('./Data/valid_data_stage1_y.npy'),
    'test_X': np.load('./Data/test_data_stage1_X.npy'),
    'test_y': np.load('./Data/test_data_stage1_y.npy'),
}

stage2 = {
    'train_X': np.load('./Data/train_data_stage2_X.npy'),
    'train_y': np.load('./Data/train_data_stage2_y.npy'),
    'valid_X': np.load('./Data/valid_data_stage2_X.npy'),
    'valid_y': np.load('./Data/valid_data_stage2_y.npy'),
    'test_X': np.load('./Data/test_data_stage2_X.npy'),
    'test_y': np.load('./Data/test_data_stage2_y.npy'),
}

columns = json.load(open('./Data/valid_columns.json', 'r'))

### 1.3 데이터 표준화하기

### 1.3.1 Stage1 데이터 표준화하기

In [3]:
stage1_X_mean = stage1['train_X'].mean(axis = 0)
stage1_y_mean = stage1['train_y'].mean(axis = 0)
print('입력값 평균:', stage1_X_mean)
print('출력값 평균:', stage1_y_mean)

입력값 평균: [  11.85294587  205.74478705  951.00255435 1242.437388     72.01273285
   72.01162996   70.34571071   11.07603585  408.95062376   81.47648118
   75.96962985   12.79330524  566.38390195  202.65058918   68.99620414
   69.10054828   73.3958155    13.89662164  226.12474952   76.81680896
   59.99898793    9.08815439  205.71384206  425.06943434  202.26546402
   78.00841235   78.00453278  345.11532917   13.26721071  246.73680272
   74.14375402   65.00868711  108.96768422   84.98831262   80.00354672
   15.32405834   23.84402174]
출력값 평균: [12.89768652 13.69662267  8.00480187 11.36113325 21.31940678 32.87843839
  0.12721732  1.34725458  1.09990652 19.8051213   7.68305334  1.4919701
  1.20588607  2.89104562 10.02515553]


In [4]:
stage1_X_std = stage1['train_X'].std(axis = 0)
stage1_y_std = stage1['train_y'].std(axis = 0)
print('입력값 표준편차:', stage1_X_std)
print('출력값 표준편차:', stage1_y_std)

입력값 표준편차: [5.10993143e-01 1.16806618e+01 1.27620111e+02 9.77321235e+01
 6.23396785e-02 4.06238919e-01 5.49867530e+00 6.36170724e-01
 2.05368437e+01 9.21642493e-01 2.06429784e+00 1.07360805e-01
 1.83380514e+01 1.48381842e+01 5.44970706e-02 1.06721153e-01
 3.94100055e-01 2.91335938e-02 3.06908107e+00 8.30358591e-01
 1.61971729e-01 3.95092636e-01 1.63223059e+01 9.53699574e+00
 1.58921785e+01 7.73526529e-02 1.14547773e-01 9.05396328e+00
 4.34061012e-01 6.10492128e+00 2.05042127e+00 6.27961350e-02
 5.59761035e+00 1.85637963e+01 1.18126539e-01 1.18770739e+00
 3.72221796e-01]
출력값 표준편차: [0.9270819  0.85503751 6.90580271 1.03664992 2.14313701 3.882343
 0.57606676 1.13060617 1.41265541 4.68687927 1.07824517 2.54294205
 0.66114934 0.92884134 7.38778856]


In [5]:
# 학습 데이터 표준화
stage1['train_X'] = (stage1['train_X'] - stage1_X_mean) / stage1_X_std
stage1['train_y'] = (stage1['train_y'] - stage1_y_mean) / stage1_y_std
# 검증용 데이터 표준화
stage1['valid_X'] = (stage1['valid_X'] - stage1_X_mean) / stage1_X_std
stage1['valid_y'] = (stage1['valid_y'] - stage1_y_mean) / stage1_y_std
# 테스트 데이터 표준화
stage1['test_X'] = (stage1['test_X'] - stage1_X_mean) / stage1_X_std
stage1['test_y'] = (stage1['test_y'] - stage1_y_mean) / stage1_y_std

### 1.3.2 Stage2 데이터 표준화하기

In [6]:
stage2_X_mean = stage2['train_X'].mean(axis = 0)
stage2_y_mean = stage2['train_y'].mean(axis = 0)
print('입력값 평균:', stage2_X_mean)
print('출력값 평균:', stage2_y_mean)

입력값 평균: [1.28976865e+01 1.36966227e+01 8.00480187e+00 1.13611333e+01
 2.13194068e+01 3.28784384e+01 1.27217323e-01 1.34725458e+00
 1.09990652e+00 1.98051213e+01 7.68305334e+00 1.49197010e+00
 1.20588607e+00 2.89104562e+00 1.00251555e+01 3.60122933e+02
 3.60136275e+02 1.72294333e+01 3.22627969e+02 3.09786854e+02
 1.87200947e+02 3.09998282e+02 2.89997582e+02 2.69681649e+02
 2.42661246e+02 2.44994629e+02 6.34195390e+01 1.54046742e+02
 1.53240583e+01 2.38440217e+01]
출력값 평균: [11.69212911  6.25614828 10.2499656  19.33555126  2.87697932  2.7475277
  0.53310706  2.91750962 18.38898671 11.61424043  7.53420311  3.54181714
  7.51699723]


In [7]:
stage2_X_std = stage2['train_X'].std(axis = 0)
stage2_y_std = stage2['train_y'].std(axis = 0)
print('입력값 표준편차:', stage2_X_std)
print('출력값 표준편차:', stage2_y_std)

입력값 표준편차: [ 0.9270819   0.85503751  6.90580271  1.03664992  2.14313701  3.882343
  0.57606676  1.13060617  1.41265541  4.68687927  1.07824517  2.54294205
  0.66114934  0.92884134  7.38778856  1.9209727   2.64895901  0.94209921
  3.61537985  2.74078134 23.41676809  0.03961971  0.05202987  1.01853204
  1.59098238  0.10771334  0.39587306  9.90869048  1.18770739  0.3722218 ]
출력값 표준편차: [3.62230517 1.60057596 2.30753588 4.62537631 9.09270122 0.37574164
 0.20467783 0.49744294 4.97294003 7.62127826 1.64901449 0.45450424
 2.0723342 ]


In [8]:
# 학습 데이터 표준화
stage2['train_X'] = (stage2['train_X'] - stage2_X_mean) / stage2_X_std
stage2['train_y'] = (stage2['train_y'] - stage2_y_mean) / stage2_y_std

# 검증용 데이터 표준화
stage2['valid_X'] = (stage2['valid_X'] - stage2_X_mean) / stage2_X_std
stage2['valid_y'] = (stage2['valid_y'] - stage2_y_mean) / stage2_y_std

# 테스트 데이터 표준화
stage2['test_X'] = (stage2['test_X'] - stage2_X_mean) / stage2_X_std
stage2['test_y'] = (stage2['test_y'] - stage2_y_mean) / stage2_y_std

### 1.4 Early stop
검증용 데이터를 이용하여 모델이 과적합 되기 전에 학습을 중지해보겠습니다.

### 1.4.1 모델 정의

In [9]:
# MLP 모델을 설정합니다.
# early stop 을 하지 않을 모델입니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])
# early stop 을 사용할 모델입니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_es = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 1.4.2 모델 학습 방법 설정

In [10]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])
MLP_model_es.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])

### 1.4.3 모델 학습

먼저, early stop 을 사용하지 않을 모델부터 학습합니다. 실습시간을 고려하여 전체 학습데이터 중 1000개만 사용하겠습니다.

In [11]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 1s - loss: 1.1605 - mse: 1.1605
Epoch 2/500
63/63 - 0s - loss: 1.1044 - mse: 1.1044
Epoch 3/500
63/63 - 0s - loss: 1.0752 - mse: 1.0752
Epoch 4/500
63/63 - 0s - loss: 1.0524 - mse: 1.0524
Epoch 5/500
63/63 - 0s - loss: 1.0320 - mse: 1.0320
Epoch 6/500
63/63 - 0s - loss: 1.0123 - mse: 1.0123
Epoch 7/500
63/63 - 0s - loss: 0.9935 - mse: 0.9935
Epoch 8/500
63/63 - 0s - loss: 0.9763 - mse: 0.9763
Epoch 9/500
63/63 - 0s - loss: 0.9609 - mse: 0.9609
Epoch 10/500
63/63 - 0s - loss: 0.9462 - mse: 0.9462
Epoch 11/500
63/63 - 0s - loss: 0.9324 - mse: 0.9324
Epoch 12/500
63/63 - 0s - loss: 0.9203 - mse: 0.9203
Epoch 13/500
63/63 - 0s - loss: 0.9085 - mse: 0.9085
Epoch 14/500
63/63 - 0s - loss: 0.8980 - mse: 0.8980
Epoch 15/500
63/63 - 0s - loss: 0.8886 - mse: 0.8886
Epoch 16/500
63/63 - 0s - loss: 0.8800 - mse: 0.8800
Epoch 17/500
63/63 - 0s - loss: 0.8723 - mse: 0.8723
Epoch 18/500
63/63 - 0s - loss: 0.8655 - mse: 0.8655
Epoch 19/500
63/63 - 0s - loss: 0.8590 - mse: 0.8590
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3062 - mse: 0.3062
Epoch 156/500
63/63 - 0s - loss: 0.3051 - mse: 0.3051
Epoch 157/500
63/63 - 0s - loss: 0.3043 - mse: 0.3043
Epoch 158/500
63/63 - 0s - loss: 0.3036 - mse: 0.3036
Epoch 159/500
63/63 - 0s - loss: 0.3028 - mse: 0.3028
Epoch 160/500
63/63 - 0s - loss: 0.3022 - mse: 0.3022
Epoch 161/500
63/63 - 0s - loss: 0.3010 - mse: 0.3010
Epoch 162/500
63/63 - 0s - loss: 0.2998 - mse: 0.2998
Epoch 163/500
63/63 - 0s - loss: 0.2994 - mse: 0.2994
Epoch 164/500
63/63 - 0s - loss: 0.2981 - mse: 0.2981
Epoch 165/500
63/63 - 0s - loss: 0.2978 - mse: 0.2978
Epoch 166/500
63/63 - 0s - loss: 0.2973 - mse: 0.2973
Epoch 167/500
63/63 - 0s - loss: 0.2962 - mse: 0.2962
Epoch 168/500
63/63 - 0s - loss: 0.2951 - mse: 0.2951
Epoch 169/500
63/63 - 0s - loss: 0.2946 - mse: 0.2946
Epoch 170/500
63/63 - 0s - loss: 0.2938 - mse: 0.2938
Epoch 171/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929
Epoch 172/500
63/63 - 0s - loss: 0.2922 - mse: 0.2922
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2174 - mse: 0.2174
Epoch 308/500
63/63 - 0s - loss: 0.2167 - mse: 0.2167
Epoch 309/500
63/63 - 0s - loss: 0.2161 - mse: 0.2161
Epoch 310/500
63/63 - 0s - loss: 0.2159 - mse: 0.2159
Epoch 311/500
63/63 - 0s - loss: 0.2150 - mse: 0.2150
Epoch 312/500
63/63 - 0s - loss: 0.2146 - mse: 0.2146
Epoch 313/500
63/63 - 0s - loss: 0.2142 - mse: 0.2142
Epoch 314/500
63/63 - 0s - loss: 0.2140 - mse: 0.2140
Epoch 315/500
63/63 - 0s - loss: 0.2136 - mse: 0.2136
Epoch 316/500
63/63 - 0s - loss: 0.2130 - mse: 0.2130
Epoch 317/500
63/63 - 0s - loss: 0.2125 - mse: 0.2125
Epoch 318/500
63/63 - 0s - loss: 0.2118 - mse: 0.2118
Epoch 319/500
63/63 - 0s - loss: 0.2116 - mse: 0.2116
Epoch 320/500
63/63 - 0s - loss: 0.2108 - mse: 0.2108
Epoch 321/500
63/63 - 0s - loss: 0.2103 - mse: 0.2103
Epoch 322/500
63/63 - 0s - loss: 0.2100 - mse: 0.2100
Epoch 323/500
63/63 - 0s - loss: 0.2092 - mse: 0.2092
Epoch 324/500
63/63 - 0s - loss: 0.2090 - mse: 0.2090
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1537 - mse: 0.1537
Epoch 460/500
63/63 - 0s - loss: 0.1531 - mse: 0.1531
Epoch 461/500
63/63 - 0s - loss: 0.1530 - mse: 0.1530
Epoch 462/500
63/63 - 0s - loss: 0.1530 - mse: 0.1530
Epoch 463/500
63/63 - 0s - loss: 0.1522 - mse: 0.1522
Epoch 464/500
63/63 - 0s - loss: 0.1516 - mse: 0.1516
Epoch 465/500
63/63 - 0s - loss: 0.1515 - mse: 0.1515
Epoch 466/500
63/63 - 0s - loss: 0.1512 - mse: 0.1512
Epoch 467/500
63/63 - 0s - loss: 0.1511 - mse: 0.1511
Epoch 468/500
63/63 - 0s - loss: 0.1504 - mse: 0.1504
Epoch 469/500
63/63 - 0s - loss: 0.1505 - mse: 0.1505
Epoch 470/500
63/63 - 0s - loss: 0.1501 - mse: 0.1501
Epoch 471/500
63/63 - 0s - loss: 0.1496 - mse: 0.1496
Epoch 472/500
63/63 - 0s - loss: 0.1494 - mse: 0.1494
Epoch 473/500
63/63 - 0s - loss: 0.1493 - mse: 0.1493
Epoch 474/500
63/63 - 0s - loss: 0.1485 - mse: 0.1485
Epoch 475/500
63/63 - 0s - loss: 0.1485 - mse: 0.1485
Epoch 476/500
63/63 - 0s - loss: 0.1478 - mse: 0.1478
Epoch 477/500
63/63 - 0s - l

다음으로, early stop을 사용할 모델을 학습합니다.

In [12]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50)
history = MLP_model_es.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], 
                           validation_data = (stage1['valid_X'], stage1['valid_y']),
                           epochs = 500, batch_size = 16, verbose = 2,
                          callbacks = [es])

Epoch 1/500
63/63 - 0s - loss: 1.1636 - mse: 1.1636 - val_loss: 1.0594 - val_mse: 1.0594
Epoch 2/500
63/63 - 0s - loss: 1.1008 - mse: 1.1008 - val_loss: 1.0168 - val_mse: 1.0168
Epoch 3/500
63/63 - 0s - loss: 1.0653 - mse: 1.0653 - val_loss: 0.9783 - val_mse: 0.9783
Epoch 4/500
63/63 - 0s - loss: 1.0309 - mse: 1.0309 - val_loss: 0.9406 - val_mse: 0.9406
Epoch 5/500
63/63 - 0s - loss: 0.9990 - mse: 0.9990 - val_loss: 0.9093 - val_mse: 0.9093
Epoch 6/500
63/63 - 0s - loss: 0.9720 - mse: 0.9720 - val_loss: 0.8854 - val_mse: 0.8854
Epoch 7/500
63/63 - 0s - loss: 0.9503 - mse: 0.9503 - val_loss: 0.8672 - val_mse: 0.8672
Epoch 8/500
63/63 - 0s - loss: 0.9329 - mse: 0.9329 - val_loss: 0.8528 - val_mse: 0.8528
Epoch 9/500
63/63 - 0s - loss: 0.9182 - mse: 0.9182 - val_loss: 0.8411 - val_mse: 0.8411
Epoch 10/500
63/63 - 0s - loss: 0.9056 - mse: 0.9056 - val_loss: 0.8318 - val_mse: 0.8318
Epoch 11/500
63/63 - 0s - loss: 0.8951 - mse: 0.8951 - val_loss: 0.8257 - val_mse: 0.8257
Epoch 12/500
63/63 

63/63 - 0s - loss: 0.4324 - mse: 0.4324 - val_loss: 0.6404 - val_mse: 0.6404
Epoch 93/500
63/63 - 0s - loss: 0.4211 - mse: 0.4211 - val_loss: 0.6330 - val_mse: 0.6330
Epoch 94/500
63/63 - 0s - loss: 0.4252 - mse: 0.4252 - val_loss: 0.6242 - val_mse: 0.6242
Epoch 95/500
63/63 - 0s - loss: 0.4115 - mse: 0.4115 - val_loss: 0.6253 - val_mse: 0.6253
Epoch 96/500
63/63 - 0s - loss: 0.4090 - mse: 0.4090 - val_loss: 0.6534 - val_mse: 0.6534
Epoch 97/500
63/63 - 0s - loss: 0.4056 - mse: 0.4056 - val_loss: 0.6293 - val_mse: 0.6293
Epoch 98/500
63/63 - 0s - loss: 0.3966 - mse: 0.3966 - val_loss: 0.6279 - val_mse: 0.6279
Epoch 99/500
63/63 - 0s - loss: 0.3923 - mse: 0.3923 - val_loss: 0.7187 - val_mse: 0.7187
Epoch 100/500
63/63 - 0s - loss: 0.3924 - mse: 0.3924 - val_loss: 0.6330 - val_mse: 0.6330
Epoch 101/500
63/63 - 0s - loss: 0.3792 - mse: 0.3792 - val_loss: 0.6306 - val_mse: 0.6306
Epoch 102/500
63/63 - 0s - loss: 0.3763 - mse: 0.3763 - val_loss: 0.6405 - val_mse: 0.6405
Epoch 103/500
63/63 

Epoch 183/500
63/63 - 0s - loss: 0.2604 - mse: 0.2604 - val_loss: 0.6168 - val_mse: 0.6168
Epoch 184/500
63/63 - 0s - loss: 0.2595 - mse: 0.2595 - val_loss: 0.6141 - val_mse: 0.6141
Epoch 185/500
63/63 - 0s - loss: 0.2588 - mse: 0.2588 - val_loss: 0.6138 - val_mse: 0.6138
Epoch 186/500
63/63 - 0s - loss: 0.2582 - mse: 0.2582 - val_loss: 0.6147 - val_mse: 0.6147
Epoch 187/500
63/63 - 0s - loss: 0.2577 - mse: 0.2577 - val_loss: 0.6138 - val_mse: 0.6138
Epoch 188/500
63/63 - 0s - loss: 0.2570 - mse: 0.2570 - val_loss: 0.6156 - val_mse: 0.6156
Epoch 189/500
63/63 - 0s - loss: 0.2563 - mse: 0.2563 - val_loss: 0.6139 - val_mse: 0.6139
Epoch 190/500
63/63 - 0s - loss: 0.2555 - mse: 0.2555 - val_loss: 0.6176 - val_mse: 0.6176
Epoch 191/500
63/63 - 0s - loss: 0.2547 - mse: 0.2547 - val_loss: 0.6173 - val_mse: 0.6173
Epoch 192/500
63/63 - 0s - loss: 0.2538 - mse: 0.2538 - val_loss: 0.6151 - val_mse: 0.6151


### 1.4.4 모델 예측 및 평가

In [13]:
pred = MLP_model.predict(stage1['test_X'])

In [14]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.293775


In [15]:
pred = MLP_model_es.predict(stage1['test_X'])

In [16]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.334291


검증용 데이터셋을 이용하여 early stop 하였을 때 모델의 성능이 더 좋습니다.

### 1.5 학습 데이터 수에 따른 모델 성능 비교

In [17]:
data_use = [0.01, 0.1, 0.5, 1]

학습데이터의 1%, 10%, 50%, 100% 를 사용한 모델의 성능을 비교해보겠습니다.

In [18]:
for ratio in data_use:
    np.random.seed(0)
    random.seed(0)
    tf.random.set_seed(0)
    # 모델 정의
    MLP_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(stage1['train_y'].shape[1])
    ])
    
    # 모델 컴파일
    MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])
    
    # 학습 데이터 개수
    total_data = len(stage1['train_X'])
    len_data = int(total_data * ratio)
    # 모델 학습
    history = MLP_model.fit(stage1['train_X'][:len_data], stage1['train_y'][:len_data], 
                            epochs = 50 * int(1 / ratio),
                            batch_size = 16, verbose = 0)
    pred = MLP_model.predict(stage1['test_X'])
    r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
    print("R2 score (학습데이터 수: %d): %f"%(len_data, r2))

R2 score (학습데이터 수: 90): -0.010257
R2 score (학습데이터 수: 901): 0.357469
R2 score (학습데이터 수: 4508): 0.511291
R2 score (학습데이터 수: 9016): 0.536478


학습데이터가 많을수록 모델의 성능이 좋은 것을 확인하였습니다.

### [TODO] Stage2에 대해 Early stop 을 적용해봅니다.

In [19]:
# MLP 모델을 설정합니다. Early stop 을 사용할 모델과 사용하지 않을 모델을 모두 정의합니다.


In [20]:
# MLP 모델을 컴파일 합니다.


In [21]:
# Early stop 을 적용하지 않을 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0007 - mse: 1.0007
Epoch 2/500
63/63 - 0s - loss: 0.8786 - mse: 0.8786
Epoch 3/500
63/63 - 0s - loss: 0.7753 - mse: 0.7753
Epoch 4/500
63/63 - 0s - loss: 0.6962 - mse: 0.6962
Epoch 5/500
63/63 - 0s - loss: 0.6453 - mse: 0.6453
Epoch 6/500
63/63 - 0s - loss: 0.6155 - mse: 0.6155
Epoch 7/500
63/63 - 0s - loss: 0.5943 - mse: 0.5943
Epoch 8/500
63/63 - 0s - loss: 0.5782 - mse: 0.5782
Epoch 9/500
63/63 - 0s - loss: 0.5644 - mse: 0.5644
Epoch 10/500
63/63 - 0s - loss: 0.5515 - mse: 0.5515
Epoch 11/500
63/63 - 0s - loss: 0.5415 - mse: 0.5415
Epoch 12/500
63/63 - 0s - loss: 0.5322 - mse: 0.5322
Epoch 13/500
63/63 - 0s - loss: 0.5234 - mse: 0.5234
Epoch 14/500
63/63 - 0s - loss: 0.5148 - mse: 0.5148
Epoch 15/500
63/63 - 0s - loss: 0.5086 - mse: 0.5086
Epoch 16/500
63/63 - 0s - loss: 0.5013 - mse: 0.5013
Epoch 17/500
63/63 - 0s - loss: 0.4955 - mse: 0.4955
Epoch 18/500
63/63 - 0s - loss: 0.4904 - mse: 0.4904
Epoch 19/500
63/63 - 0s - loss: 0.4858 - mse: 0.4858
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3337 - mse: 0.3337
Epoch 156/500
63/63 - 0s - loss: 0.3329 - mse: 0.3329
Epoch 157/500
63/63 - 0s - loss: 0.3332 - mse: 0.3332
Epoch 158/500
63/63 - 0s - loss: 0.3326 - mse: 0.3326
Epoch 159/500
63/63 - 0s - loss: 0.3315 - mse: 0.3315
Epoch 160/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 161/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 162/500
63/63 - 0s - loss: 0.3299 - mse: 0.3299
Epoch 163/500
63/63 - 0s - loss: 0.3292 - mse: 0.3292
Epoch 164/500
63/63 - 0s - loss: 0.3286 - mse: 0.3286
Epoch 165/500
63/63 - 0s - loss: 0.3283 - mse: 0.3283
Epoch 166/500
63/63 - 0s - loss: 0.3276 - mse: 0.3276
Epoch 167/500
63/63 - 0s - loss: 0.3276 - mse: 0.3276
Epoch 168/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 169/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 170/500
63/63 - 0s - loss: 0.3248 - mse: 0.3248
Epoch 171/500
63/63 - 0s - loss: 0.3263 - mse: 0.3263
Epoch 172/500
63/63 - 0s - loss: 0.3232 - mse: 0.3232
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2814 - mse: 0.2814
Epoch 308/500
63/63 - 0s - loss: 0.2809 - mse: 0.2809
Epoch 309/500
63/63 - 0s - loss: 0.2815 - mse: 0.2815
Epoch 310/500
63/63 - 0s - loss: 0.2804 - mse: 0.2804
Epoch 311/500
63/63 - 0s - loss: 0.2799 - mse: 0.2799
Epoch 312/500
63/63 - 0s - loss: 0.2795 - mse: 0.2795
Epoch 313/500
63/63 - 0s - loss: 0.2799 - mse: 0.2799
Epoch 314/500
63/63 - 0s - loss: 0.2796 - mse: 0.2796
Epoch 315/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 316/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 317/500
63/63 - 0s - loss: 0.2784 - mse: 0.2784
Epoch 318/500
63/63 - 0s - loss: 0.2774 - mse: 0.2774
Epoch 319/500
63/63 - 0s - loss: 0.2789 - mse: 0.2789
Epoch 320/500
63/63 - 0s - loss: 0.2779 - mse: 0.2779
Epoch 321/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 322/500
63/63 - 0s - loss: 0.2787 - mse: 0.2787
Epoch 323/500
63/63 - 0s - loss: 0.2781 - mse: 0.2781
Epoch 324/500
63/63 - 0s - loss: 0.2781 - mse: 0.2781
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2525 - mse: 0.2525
Epoch 460/500
63/63 - 0s - loss: 0.2514 - mse: 0.2514
Epoch 461/500
63/63 - 0s - loss: 0.2520 - mse: 0.2520
Epoch 462/500
63/63 - 0s - loss: 0.2532 - mse: 0.2532
Epoch 463/500
63/63 - 0s - loss: 0.2524 - mse: 0.2524
Epoch 464/500
63/63 - 0s - loss: 0.2511 - mse: 0.2511
Epoch 465/500
63/63 - 0s - loss: 0.2514 - mse: 0.2514
Epoch 466/500
63/63 - 0s - loss: 0.2518 - mse: 0.2518
Epoch 467/500
63/63 - 0s - loss: 0.2504 - mse: 0.2504
Epoch 468/500
63/63 - 0s - loss: 0.2508 - mse: 0.2508
Epoch 469/500
63/63 - 0s - loss: 0.2502 - mse: 0.2502
Epoch 470/500
63/63 - 0s - loss: 0.2512 - mse: 0.2512
Epoch 471/500
63/63 - 0s - loss: 0.2506 - mse: 0.2506
Epoch 472/500
63/63 - 0s - loss: 0.2500 - mse: 0.2500
Epoch 473/500
63/63 - 0s - loss: 0.2498 - mse: 0.2498
Epoch 474/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 475/500
63/63 - 0s - loss: 0.2497 - mse: 0.2497
Epoch 476/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 477/500
63/63 - 0s - l

In [22]:
# 다음으로, Early stop 을 적용할 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0054 - mse: 1.0054 - val_loss: 1.0671 - val_mse: 1.0671
Epoch 2/500
63/63 - 0s - loss: 0.8761 - mse: 0.8761 - val_loss: 0.9237 - val_mse: 0.9237
Epoch 3/500
63/63 - 0s - loss: 0.7792 - mse: 0.7792 - val_loss: 0.8032 - val_mse: 0.8032
Epoch 4/500
63/63 - 0s - loss: 0.7081 - mse: 0.7081 - val_loss: 0.7341 - val_mse: 0.7341
Epoch 5/500
63/63 - 0s - loss: 0.6626 - mse: 0.6626 - val_loss: 0.6945 - val_mse: 0.6945
Epoch 6/500
63/63 - 0s - loss: 0.6361 - mse: 0.6361 - val_loss: 0.6779 - val_mse: 0.6779
Epoch 7/500
63/63 - 0s - loss: 0.6173 - mse: 0.6173 - val_loss: 0.6604 - val_mse: 0.6604
Epoch 8/500
63/63 - 0s - loss: 0.6025 - mse: 0.6025 - val_loss: 0.6504 - val_mse: 0.6504
Epoch 9/500
63/63 - 0s - loss: 0.5914 - mse: 0.5914 - val_loss: 0.6393 - val_mse: 0.6393
Epoch 10/500
63/63 - 0s - loss: 0.5793 - mse: 0.5793 - val_loss: 0.6290 - val_mse: 0.6290
Epoch 11/500
63/63 - 0s - loss: 0.5703 - mse: 0.5703 - val_loss: 0.6189 - val_mse: 0.6189
Epoch 12/500
63/63 

63/63 - 0s - loss: 0.3896 - mse: 0.3896 - val_loss: 0.4645 - val_mse: 0.4645
Epoch 93/500
63/63 - 0s - loss: 0.3875 - mse: 0.3875 - val_loss: 0.4688 - val_mse: 0.4688
Epoch 94/500
63/63 - 0s - loss: 0.3882 - mse: 0.3882 - val_loss: 0.4651 - val_mse: 0.4651
Epoch 95/500
63/63 - 0s - loss: 0.3868 - mse: 0.3868 - val_loss: 0.4641 - val_mse: 0.4641
Epoch 96/500
63/63 - 0s - loss: 0.3858 - mse: 0.3858 - val_loss: 0.4592 - val_mse: 0.4592
Epoch 97/500
63/63 - 0s - loss: 0.3855 - mse: 0.3855 - val_loss: 0.4644 - val_mse: 0.4644
Epoch 98/500
63/63 - 0s - loss: 0.3842 - mse: 0.3842 - val_loss: 0.4602 - val_mse: 0.4602
Epoch 99/500
63/63 - 0s - loss: 0.3824 - mse: 0.3824 - val_loss: 0.4581 - val_mse: 0.4581
Epoch 100/500
63/63 - 0s - loss: 0.3826 - mse: 0.3826 - val_loss: 0.4603 - val_mse: 0.4603
Epoch 101/500
63/63 - 0s - loss: 0.3819 - mse: 0.3819 - val_loss: 0.4561 - val_mse: 0.4561
Epoch 102/500
63/63 - 0s - loss: 0.3810 - mse: 0.3810 - val_loss: 0.4584 - val_mse: 0.4584
Epoch 103/500
63/63 

Epoch 183/500
63/63 - 0s - loss: 0.3287 - mse: 0.3287 - val_loss: 0.4209 - val_mse: 0.4209
Epoch 184/500
63/63 - 0s - loss: 0.3274 - mse: 0.3274 - val_loss: 0.4279 - val_mse: 0.4279
Epoch 185/500
63/63 - 0s - loss: 0.3260 - mse: 0.3260 - val_loss: 0.4252 - val_mse: 0.4252
Epoch 186/500
63/63 - 0s - loss: 0.3267 - mse: 0.3267 - val_loss: 0.4262 - val_mse: 0.4262
Epoch 187/500
63/63 - 0s - loss: 0.3264 - mse: 0.3264 - val_loss: 0.4218 - val_mse: 0.4218
Epoch 188/500
63/63 - 0s - loss: 0.3254 - mse: 0.3254 - val_loss: 0.4226 - val_mse: 0.4226
Epoch 189/500
63/63 - 0s - loss: 0.3255 - mse: 0.3255 - val_loss: 0.4255 - val_mse: 0.4255
Epoch 190/500
63/63 - 0s - loss: 0.3243 - mse: 0.3243 - val_loss: 0.4263 - val_mse: 0.4263
Epoch 191/500
63/63 - 0s - loss: 0.3247 - mse: 0.3247 - val_loss: 0.4603 - val_mse: 0.4603
Epoch 192/500
63/63 - 0s - loss: 0.3275 - mse: 0.3275 - val_loss: 0.4229 - val_mse: 0.4229
Epoch 193/500
63/63 - 0s - loss: 0.3232 - mse: 0.3232 - val_loss: 0.4255 - val_mse: 0.4255

63/63 - 0s - loss: 0.2937 - mse: 0.2937 - val_loss: 0.4217 - val_mse: 0.4217
Epoch 274/500
63/63 - 0s - loss: 0.2947 - mse: 0.2947 - val_loss: 0.4222 - val_mse: 0.4222
Epoch 275/500
63/63 - 0s - loss: 0.2941 - mse: 0.2941 - val_loss: 0.4216 - val_mse: 0.4216
Epoch 276/500
63/63 - 0s - loss: 0.2931 - mse: 0.2931 - val_loss: 0.4281 - val_mse: 0.4281
Epoch 277/500
63/63 - 0s - loss: 0.2935 - mse: 0.2935 - val_loss: 0.4272 - val_mse: 0.4272
Epoch 278/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929 - val_loss: 0.4226 - val_mse: 0.4226
Epoch 279/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929 - val_loss: 0.4189 - val_mse: 0.4189
Epoch 280/500
63/63 - 0s - loss: 0.2931 - mse: 0.2931 - val_loss: 0.4245 - val_mse: 0.4245
Epoch 281/500
63/63 - 0s - loss: 0.2907 - mse: 0.2907 - val_loss: 0.4308 - val_mse: 0.4308
Epoch 282/500
63/63 - 0s - loss: 0.2935 - mse: 0.2935 - val_loss: 0.4270 - val_mse: 0.4270
Epoch 283/500
63/63 - 0s - loss: 0.2924 - mse: 0.2924 - val_loss: 0.4206 - val_mse: 0.4206
Epoch 284/500

In [23]:
# Early stop 을 사용하지 않은 모델을 평가합니다.


R2 score: 0.587718


In [24]:
# Early stop 을 사용한 모델을 평가합니다.


R2 score: 0.451157


Stage2 의 데이터에 대해서는 early stop을 사용하였을 때, r2-score 가 더 낮은 것을 확인할 수 있습니다. 모든 데이터셋에 대해 특정 방법이 항상 우세하지는 않습니다. 데이터마다 어떠한 방법을 선택하고, 어떤 하이퍼 파라미터를 선택할지 신중하게 선택할 필요가 있습니다.

## 2. 가중치 규제
---
금속분말 데이터셋을 이용하여 가중치 규제를 모델에 적용해보겠습니다.


### 2.1 모델 정의

In [25]:
# MLP 모델을 설정합니다.
# 가중치 규제를 하지 않을 모델입니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 가중치규제를 사용할 모델입니다. L1 regularizer을 적용해봅니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_reg = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)), 
    tf.keras.layers.Dense(64, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(32, activation = 'relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

## 2.2 모델 학습 방법 설정

In [26]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])
MLP_model_reg.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])

## 2.3 모델 학습

In [27]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.1605 - mse: 1.1605
Epoch 2/500
63/63 - 0s - loss: 1.1044 - mse: 1.1044
Epoch 3/500
63/63 - 0s - loss: 1.0752 - mse: 1.0752
Epoch 4/500
63/63 - 0s - loss: 1.0524 - mse: 1.0524
Epoch 5/500
63/63 - 0s - loss: 1.0320 - mse: 1.0320
Epoch 6/500
63/63 - 0s - loss: 1.0123 - mse: 1.0123
Epoch 7/500
63/63 - 0s - loss: 0.9935 - mse: 0.9935
Epoch 8/500
63/63 - 0s - loss: 0.9763 - mse: 0.9763
Epoch 9/500
63/63 - 0s - loss: 0.9609 - mse: 0.9609
Epoch 10/500
63/63 - 0s - loss: 0.9462 - mse: 0.9462
Epoch 11/500
63/63 - 0s - loss: 0.9324 - mse: 0.9324
Epoch 12/500
63/63 - 0s - loss: 0.9203 - mse: 0.9203
Epoch 13/500
63/63 - 0s - loss: 0.9085 - mse: 0.9085
Epoch 14/500
63/63 - 0s - loss: 0.8980 - mse: 0.8980
Epoch 15/500
63/63 - 0s - loss: 0.8886 - mse: 0.8886
Epoch 16/500
63/63 - 0s - loss: 0.8800 - mse: 0.8800
Epoch 17/500
63/63 - 0s - loss: 0.8723 - mse: 0.8723
Epoch 18/500
63/63 - 0s - loss: 0.8655 - mse: 0.8655
Epoch 19/500
63/63 - 0s - loss: 0.8590 - mse: 0.8590
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3062 - mse: 0.3062
Epoch 156/500
63/63 - 0s - loss: 0.3051 - mse: 0.3051
Epoch 157/500
63/63 - 0s - loss: 0.3043 - mse: 0.3043
Epoch 158/500
63/63 - 0s - loss: 0.3036 - mse: 0.3036
Epoch 159/500
63/63 - 0s - loss: 0.3028 - mse: 0.3028
Epoch 160/500
63/63 - 0s - loss: 0.3022 - mse: 0.3022
Epoch 161/500
63/63 - 0s - loss: 0.3010 - mse: 0.3010
Epoch 162/500
63/63 - 0s - loss: 0.2998 - mse: 0.2998
Epoch 163/500
63/63 - 0s - loss: 0.2994 - mse: 0.2994
Epoch 164/500
63/63 - 0s - loss: 0.2981 - mse: 0.2981
Epoch 165/500
63/63 - 0s - loss: 0.2978 - mse: 0.2978
Epoch 166/500
63/63 - 0s - loss: 0.2973 - mse: 0.2973
Epoch 167/500
63/63 - 0s - loss: 0.2962 - mse: 0.2962
Epoch 168/500
63/63 - 0s - loss: 0.2951 - mse: 0.2951
Epoch 169/500
63/63 - 0s - loss: 0.2946 - mse: 0.2946
Epoch 170/500
63/63 - 0s - loss: 0.2938 - mse: 0.2938
Epoch 171/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929
Epoch 172/500
63/63 - 0s - loss: 0.2922 - mse: 0.2922
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2173 - mse: 0.2173
Epoch 308/500
63/63 - 0s - loss: 0.2167 - mse: 0.2167
Epoch 309/500
63/63 - 0s - loss: 0.2161 - mse: 0.2161
Epoch 310/500
63/63 - 0s - loss: 0.2159 - mse: 0.2159
Epoch 311/500
63/63 - 0s - loss: 0.2150 - mse: 0.2150
Epoch 312/500
63/63 - 0s - loss: 0.2146 - mse: 0.2146
Epoch 313/500
63/63 - 0s - loss: 0.2142 - mse: 0.2142
Epoch 314/500
63/63 - 0s - loss: 0.2140 - mse: 0.2140
Epoch 315/500
63/63 - 0s - loss: 0.2136 - mse: 0.2136
Epoch 316/500
63/63 - 0s - loss: 0.2130 - mse: 0.2130
Epoch 317/500
63/63 - 0s - loss: 0.2125 - mse: 0.2125
Epoch 318/500
63/63 - 0s - loss: 0.2118 - mse: 0.2118
Epoch 319/500
63/63 - 0s - loss: 0.2116 - mse: 0.2116
Epoch 320/500
63/63 - 0s - loss: 0.2107 - mse: 0.2107
Epoch 321/500
63/63 - 0s - loss: 0.2103 - mse: 0.2103
Epoch 322/500
63/63 - 0s - loss: 0.2100 - mse: 0.2100
Epoch 323/500
63/63 - 0s - loss: 0.2092 - mse: 0.2092
Epoch 324/500
63/63 - 0s - loss: 0.2090 - mse: 0.2090
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1537 - mse: 0.1537
Epoch 460/500
63/63 - 0s - loss: 0.1531 - mse: 0.1531
Epoch 461/500
63/63 - 0s - loss: 0.1529 - mse: 0.1529
Epoch 462/500
63/63 - 0s - loss: 0.1530 - mse: 0.1530
Epoch 463/500
63/63 - 0s - loss: 0.1522 - mse: 0.1522
Epoch 464/500
63/63 - 0s - loss: 0.1515 - mse: 0.1515
Epoch 465/500
63/63 - 0s - loss: 0.1514 - mse: 0.1514
Epoch 466/500
63/63 - 0s - loss: 0.1511 - mse: 0.1511
Epoch 467/500
63/63 - 0s - loss: 0.1510 - mse: 0.1510
Epoch 468/500
63/63 - 0s - loss: 0.1503 - mse: 0.1503
Epoch 469/500
63/63 - 0s - loss: 0.1505 - mse: 0.1505
Epoch 470/500
63/63 - 0s - loss: 0.1500 - mse: 0.1500
Epoch 471/500
63/63 - 0s - loss: 0.1494 - mse: 0.1494
Epoch 472/500
63/63 - 0s - loss: 0.1493 - mse: 0.1493
Epoch 473/500
63/63 - 0s - loss: 0.1491 - mse: 0.1491
Epoch 474/500
63/63 - 0s - loss: 0.1484 - mse: 0.1484
Epoch 475/500
63/63 - 0s - loss: 0.1484 - mse: 0.1484
Epoch 476/500
63/63 - 0s - loss: 0.1476 - mse: 0.1476
Epoch 477/500
63/63 - 0s - l

In [28]:
history = MLP_model_reg.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.3490 - mse: 1.1635
Epoch 2/500
63/63 - 0s - loss: 1.2856 - mse: 1.1008
Epoch 3/500
63/63 - 0s - loss: 1.2497 - mse: 1.0655
Epoch 4/500
63/63 - 0s - loss: 1.2153 - mse: 1.0314
Epoch 5/500
63/63 - 0s - loss: 1.1833 - mse: 0.9997
Epoch 6/500
63/63 - 0s - loss: 1.1561 - mse: 0.9728
Epoch 7/500
63/63 - 0s - loss: 1.1342 - mse: 0.9512
Epoch 8/500
63/63 - 0s - loss: 1.1165 - mse: 0.9339
Epoch 9/500
63/63 - 0s - loss: 1.1015 - mse: 0.9192
Epoch 10/500
63/63 - 0s - loss: 1.0886 - mse: 0.9066
Epoch 11/500
63/63 - 0s - loss: 1.0779 - mse: 0.8962
Epoch 12/500
63/63 - 0s - loss: 1.0686 - mse: 0.8872
Epoch 13/500
63/63 - 0s - loss: 1.0597 - mse: 0.8787
Epoch 14/500
63/63 - 0s - loss: 1.0517 - mse: 0.8710
Epoch 15/500
63/63 - 0s - loss: 1.0444 - mse: 0.8640
Epoch 16/500
63/63 - 0s - loss: 1.0375 - mse: 0.8575
Epoch 17/500
63/63 - 0s - loss: 1.0310 - mse: 0.8514
Epoch 18/500
63/63 - 0s - loss: 1.0248 - mse: 0.8454
Epoch 19/500
63/63 - 0s - loss: 1.0187 - mse: 0.8397
Ep

Epoch 155/500
63/63 - 0s - loss: 0.4695 - mse: 0.3032
Epoch 156/500
63/63 - 0s - loss: 0.4683 - mse: 0.3022
Epoch 157/500
63/63 - 0s - loss: 0.4670 - mse: 0.3011
Epoch 158/500
63/63 - 0s - loss: 0.4667 - mse: 0.3010
Epoch 159/500
63/63 - 0s - loss: 0.4653 - mse: 0.2998
Epoch 160/500
63/63 - 0s - loss: 0.4649 - mse: 0.2995
Epoch 161/500
63/63 - 0s - loss: 0.4629 - mse: 0.2977
Epoch 162/500
63/63 - 0s - loss: 0.4624 - mse: 0.2974
Epoch 163/500
63/63 - 0s - loss: 0.4617 - mse: 0.2968
Epoch 164/500
63/63 - 0s - loss: 0.4597 - mse: 0.2950
Epoch 165/500
63/63 - 0s - loss: 0.4602 - mse: 0.2957
Epoch 166/500
63/63 - 0s - loss: 0.4600 - mse: 0.2955
Epoch 167/500
63/63 - 0s - loss: 0.4585 - mse: 0.2942
Epoch 168/500
63/63 - 0s - loss: 0.4569 - mse: 0.2928
Epoch 169/500
63/63 - 0s - loss: 0.4563 - mse: 0.2924
Epoch 170/500
63/63 - 0s - loss: 0.4553 - mse: 0.2916
Epoch 171/500
63/63 - 0s - loss: 0.4544 - mse: 0.2908
Epoch 172/500
63/63 - 0s - loss: 0.4538 - mse: 0.2905
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.3768 - mse: 0.2323
Epoch 308/500
63/63 - 0s - loss: 0.3765 - mse: 0.2322
Epoch 309/500
63/63 - 0s - loss: 0.3760 - mse: 0.2318
Epoch 310/500
63/63 - 0s - loss: 0.3756 - mse: 0.2315
Epoch 311/500
63/63 - 0s - loss: 0.3751 - mse: 0.2311
Epoch 312/500
63/63 - 0s - loss: 0.3742 - mse: 0.2304
Epoch 313/500
63/63 - 0s - loss: 0.3743 - mse: 0.2306
Epoch 314/500
63/63 - 0s - loss: 0.3738 - mse: 0.2302
Epoch 315/500
63/63 - 0s - loss: 0.3734 - mse: 0.2298
Epoch 316/500
63/63 - 0s - loss: 0.3726 - mse: 0.2291
Epoch 317/500
63/63 - 0s - loss: 0.3725 - mse: 0.2291
Epoch 318/500
63/63 - 0s - loss: 0.3718 - mse: 0.2285
Epoch 319/500
63/63 - 0s - loss: 0.3713 - mse: 0.2281
Epoch 320/500
63/63 - 0s - loss: 0.3710 - mse: 0.2279
Epoch 321/500
63/63 - 0s - loss: 0.3707 - mse: 0.2277
Epoch 322/500
63/63 - 0s - loss: 0.3700 - mse: 0.2271
Epoch 323/500
63/63 - 0s - loss: 0.3694 - mse: 0.2266
Epoch 324/500
63/63 - 0s - loss: 0.3692 - mse: 0.2265
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.3186 - mse: 0.1861
Epoch 460/500
63/63 - 0s - loss: 0.3182 - mse: 0.1858
Epoch 461/500
63/63 - 0s - loss: 0.3183 - mse: 0.1860
Epoch 462/500
63/63 - 0s - loss: 0.3182 - mse: 0.1859
Epoch 463/500
63/63 - 0s - loss: 0.3178 - mse: 0.1855
Epoch 464/500
63/63 - 0s - loss: 0.3172 - mse: 0.1850
Epoch 465/500
63/63 - 0s - loss: 0.3168 - mse: 0.1847
Epoch 466/500
63/63 - 0s - loss: 0.3166 - mse: 0.1846
Epoch 467/500
63/63 - 0s - loss: 0.3162 - mse: 0.1842
Epoch 468/500
63/63 - 0s - loss: 0.3160 - mse: 0.1841
Epoch 469/500
63/63 - 0s - loss: 0.3158 - mse: 0.1840
Epoch 470/500
63/63 - 0s - loss: 0.3160 - mse: 0.1842
Epoch 471/500
63/63 - 0s - loss: 0.3153 - mse: 0.1835
Epoch 472/500
63/63 - 0s - loss: 0.3151 - mse: 0.1834
Epoch 473/500
63/63 - 0s - loss: 0.3145 - mse: 0.1828
Epoch 474/500
63/63 - 0s - loss: 0.3137 - mse: 0.1821
Epoch 475/500
63/63 - 0s - loss: 0.3140 - mse: 0.1825
Epoch 476/500
63/63 - 0s - loss: 0.3132 - mse: 0.1817
Epoch 477/500
63/63 - 0s - l

In [29]:
pred = MLP_model.predict(stage1['test_X'])

In [30]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.294210


In [31]:
pred = MLP_model_reg.predict(stage1['test_X'])

In [32]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.340328


가중치 규제를 사용한 모델에서 성능이 더 좋았습니다.

### [TODO] Stage2에 대해 가중치규제를 적용해봅니다.

In [33]:
# MLP 모델을 설정합니다. 가중치규제를 사용할 모델과 사용하지 않을 모델을 모두 정의합니다.


In [34]:
# MLP 모델을 컴파일 합니다.


In [35]:
# 가중치규제를 사용하지 않는 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0007 - mse: 1.0007
Epoch 2/500
63/63 - 0s - loss: 0.8786 - mse: 0.8786
Epoch 3/500
63/63 - 0s - loss: 0.7753 - mse: 0.7753
Epoch 4/500
63/63 - 0s - loss: 0.6962 - mse: 0.6962
Epoch 5/500
63/63 - 0s - loss: 0.6453 - mse: 0.6453
Epoch 6/500
63/63 - 0s - loss: 0.6155 - mse: 0.6155
Epoch 7/500
63/63 - 0s - loss: 0.5943 - mse: 0.5943
Epoch 8/500
63/63 - 0s - loss: 0.5782 - mse: 0.5782
Epoch 9/500
63/63 - 0s - loss: 0.5644 - mse: 0.5644
Epoch 10/500
63/63 - 0s - loss: 0.5515 - mse: 0.5515
Epoch 11/500
63/63 - 0s - loss: 0.5415 - mse: 0.5415
Epoch 12/500
63/63 - 0s - loss: 0.5322 - mse: 0.5322
Epoch 13/500
63/63 - 0s - loss: 0.5234 - mse: 0.5234
Epoch 14/500
63/63 - 0s - loss: 0.5148 - mse: 0.5148
Epoch 15/500
63/63 - 0s - loss: 0.5086 - mse: 0.5086
Epoch 16/500
63/63 - 0s - loss: 0.5013 - mse: 0.5013
Epoch 17/500
63/63 - 0s - loss: 0.4955 - mse: 0.4955
Epoch 18/500
63/63 - 0s - loss: 0.4904 - mse: 0.4904
Epoch 19/500
63/63 - 0s - loss: 0.4858 - mse: 0.4858
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3337 - mse: 0.3337
Epoch 156/500
63/63 - 0s - loss: 0.3329 - mse: 0.3329
Epoch 157/500
63/63 - 0s - loss: 0.3331 - mse: 0.3331
Epoch 158/500
63/63 - 0s - loss: 0.3326 - mse: 0.3326
Epoch 159/500
63/63 - 0s - loss: 0.3316 - mse: 0.3316
Epoch 160/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 161/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 162/500
63/63 - 0s - loss: 0.3299 - mse: 0.3299
Epoch 163/500
63/63 - 0s - loss: 0.3292 - mse: 0.3292
Epoch 164/500
63/63 - 0s - loss: 0.3286 - mse: 0.3286
Epoch 165/500
63/63 - 0s - loss: 0.3282 - mse: 0.3282
Epoch 166/500
63/63 - 0s - loss: 0.3276 - mse: 0.3276
Epoch 167/500
63/63 - 0s - loss: 0.3276 - mse: 0.3276
Epoch 168/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 169/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 170/500
63/63 - 0s - loss: 0.3248 - mse: 0.3248
Epoch 171/500
63/63 - 0s - loss: 0.3264 - mse: 0.3264
Epoch 172/500
63/63 - 0s - loss: 0.3232 - mse: 0.3232
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2813 - mse: 0.2813
Epoch 308/500
63/63 - 0s - loss: 0.2809 - mse: 0.2809
Epoch 309/500
63/63 - 0s - loss: 0.2814 - mse: 0.2814
Epoch 310/500
63/63 - 0s - loss: 0.2804 - mse: 0.2804
Epoch 311/500
63/63 - 0s - loss: 0.2797 - mse: 0.2797
Epoch 312/500
63/63 - 0s - loss: 0.2795 - mse: 0.2795
Epoch 313/500
63/63 - 0s - loss: 0.2798 - mse: 0.2798
Epoch 314/500
63/63 - 0s - loss: 0.2796 - mse: 0.2796
Epoch 315/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 316/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 317/500
63/63 - 0s - loss: 0.2784 - mse: 0.2784
Epoch 318/500
63/63 - 0s - loss: 0.2773 - mse: 0.2773
Epoch 319/500
63/63 - 0s - loss: 0.2789 - mse: 0.2789
Epoch 320/500
63/63 - 0s - loss: 0.2781 - mse: 0.2781
Epoch 321/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 322/500
63/63 - 0s - loss: 0.2786 - mse: 0.2786
Epoch 323/500
63/63 - 0s - loss: 0.2782 - mse: 0.2782
Epoch 324/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2525 - mse: 0.2525
Epoch 460/500
63/63 - 0s - loss: 0.2514 - mse: 0.2514
Epoch 461/500
63/63 - 0s - loss: 0.2519 - mse: 0.2519
Epoch 462/500
63/63 - 0s - loss: 0.2534 - mse: 0.2534
Epoch 463/500
63/63 - 0s - loss: 0.2525 - mse: 0.2525
Epoch 464/500
63/63 - 0s - loss: 0.2511 - mse: 0.2511
Epoch 465/500
63/63 - 0s - loss: 0.2515 - mse: 0.2515
Epoch 466/500
63/63 - 0s - loss: 0.2518 - mse: 0.2518
Epoch 467/500
63/63 - 0s - loss: 0.2505 - mse: 0.2505
Epoch 468/500
63/63 - 0s - loss: 0.2507 - mse: 0.2507
Epoch 469/500
63/63 - 0s - loss: 0.2502 - mse: 0.2502
Epoch 470/500
63/63 - 0s - loss: 0.2509 - mse: 0.2509
Epoch 471/500
63/63 - 0s - loss: 0.2508 - mse: 0.2508
Epoch 472/500
63/63 - 0s - loss: 0.2500 - mse: 0.2500
Epoch 473/500
63/63 - 0s - loss: 0.2499 - mse: 0.2499
Epoch 474/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 475/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 476/500
63/63 - 0s - loss: 0.2497 - mse: 0.2497
Epoch 477/500
63/63 - 0s - l

In [36]:
# 다음으로, 가중치규제를 사용하는 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.1818 - mse: 1.0054
Epoch 2/500
63/63 - 0s - loss: 1.0524 - mse: 0.8764
Epoch 3/500
63/63 - 0s - loss: 0.9557 - mse: 0.7799
Epoch 4/500
63/63 - 0s - loss: 0.8845 - mse: 0.7089
Epoch 5/500
63/63 - 0s - loss: 0.8386 - mse: 0.6633
Epoch 6/500
63/63 - 0s - loss: 0.8119 - mse: 0.6368
Epoch 7/500
63/63 - 0s - loss: 0.7929 - mse: 0.6181
Epoch 8/500
63/63 - 0s - loss: 0.7778 - mse: 0.6034
Epoch 9/500
63/63 - 0s - loss: 0.7667 - mse: 0.5925
Epoch 10/500
63/63 - 0s - loss: 0.7545 - mse: 0.5807
Epoch 11/500
63/63 - 0s - loss: 0.7452 - mse: 0.5718
Epoch 12/500
63/63 - 0s - loss: 0.7362 - mse: 0.5630
Epoch 13/500
63/63 - 0s - loss: 0.7278 - mse: 0.5548
Epoch 14/500
63/63 - 0s - loss: 0.7189 - mse: 0.5462
Epoch 15/500
63/63 - 0s - loss: 0.7120 - mse: 0.5396
Epoch 16/500
63/63 - 0s - loss: 0.7043 - mse: 0.5322
Epoch 17/500
63/63 - 0s - loss: 0.6976 - mse: 0.5258
Epoch 18/500
63/63 - 0s - loss: 0.6914 - mse: 0.5199
Epoch 19/500
63/63 - 0s - loss: 0.6854 - mse: 0.5142
Ep

Epoch 155/500
63/63 - 0s - loss: 0.5006 - mse: 0.3619
Epoch 156/500
63/63 - 0s - loss: 0.5001 - mse: 0.3616
Epoch 157/500
63/63 - 0s - loss: 0.5005 - mse: 0.3622
Epoch 158/500
63/63 - 0s - loss: 0.4993 - mse: 0.3612
Epoch 159/500
63/63 - 0s - loss: 0.4985 - mse: 0.3606
Epoch 160/500
63/63 - 0s - loss: 0.4971 - mse: 0.3594
Epoch 161/500
63/63 - 0s - loss: 0.4968 - mse: 0.3592
Epoch 162/500
63/63 - 0s - loss: 0.4967 - mse: 0.3593
Epoch 163/500
63/63 - 0s - loss: 0.4958 - mse: 0.3585
Epoch 164/500
63/63 - 0s - loss: 0.4951 - mse: 0.3580
Epoch 165/500
63/63 - 0s - loss: 0.4944 - mse: 0.3575
Epoch 166/500
63/63 - 0s - loss: 0.4940 - mse: 0.3573
Epoch 167/500
63/63 - 0s - loss: 0.4935 - mse: 0.3570
Epoch 168/500
63/63 - 0s - loss: 0.4927 - mse: 0.3564
Epoch 169/500
63/63 - 0s - loss: 0.4924 - mse: 0.3562
Epoch 170/500
63/63 - 0s - loss: 0.4905 - mse: 0.3544
Epoch 171/500
63/63 - 0s - loss: 0.4915 - mse: 0.3557
Epoch 172/500
63/63 - 0s - loss: 0.4892 - mse: 0.3535
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.4298 - mse: 0.3144
Epoch 308/500
63/63 - 0s - loss: 0.4293 - mse: 0.3141
Epoch 309/500
63/63 - 0s - loss: 0.4293 - mse: 0.3141
Epoch 310/500
63/63 - 0s - loss: 0.4288 - mse: 0.3138
Epoch 311/500
63/63 - 0s - loss: 0.4285 - mse: 0.3136
Epoch 312/500
63/63 - 0s - loss: 0.4268 - mse: 0.3121
Epoch 313/500
63/63 - 0s - loss: 0.4279 - mse: 0.3132
Epoch 314/500
63/63 - 0s - loss: 0.4272 - mse: 0.3127
Epoch 315/500
63/63 - 0s - loss: 0.4262 - mse: 0.3118
Epoch 316/500
63/63 - 0s - loss: 0.4267 - mse: 0.3125
Epoch 317/500
63/63 - 0s - loss: 0.4254 - mse: 0.3112
Epoch 318/500
63/63 - 0s - loss: 0.4246 - mse: 0.3105
Epoch 319/500
63/63 - 0s - loss: 0.4256 - mse: 0.3117
Epoch 320/500
63/63 - 0s - loss: 0.4255 - mse: 0.3117
Epoch 321/500
63/63 - 0s - loss: 0.4247 - mse: 0.3110
Epoch 322/500
63/63 - 0s - loss: 0.4262 - mse: 0.3126
Epoch 323/500
63/63 - 0s - loss: 0.4251 - mse: 0.3117
Epoch 324/500
63/63 - 0s - loss: 0.4249 - mse: 0.3116
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.3912 - mse: 0.2920
Epoch 460/500
63/63 - 0s - loss: 0.3906 - mse: 0.2915
Epoch 461/500
63/63 - 0s - loss: 0.3908 - mse: 0.2918
Epoch 462/500
63/63 - 0s - loss: 0.3910 - mse: 0.2921
Epoch 463/500
63/63 - 0s - loss: 0.3909 - mse: 0.2921
Epoch 464/500
63/63 - 0s - loss: 0.3892 - mse: 0.2904
Epoch 465/500
63/63 - 0s - loss: 0.3900 - mse: 0.2914
Epoch 466/500
63/63 - 0s - loss: 0.3899 - mse: 0.2913
Epoch 467/500
63/63 - 0s - loss: 0.3892 - mse: 0.2908
Epoch 468/500
63/63 - 0s - loss: 0.3888 - mse: 0.2904
Epoch 469/500
63/63 - 0s - loss: 0.3889 - mse: 0.2906
Epoch 470/500
63/63 - 0s - loss: 0.3886 - mse: 0.2903
Epoch 471/500
63/63 - 0s - loss: 0.3884 - mse: 0.2902
Epoch 472/500
63/63 - 0s - loss: 0.3886 - mse: 0.2905
Epoch 473/500
63/63 - 0s - loss: 0.3880 - mse: 0.2900
Epoch 474/500
63/63 - 0s - loss: 0.3880 - mse: 0.2901
Epoch 475/500
63/63 - 0s - loss: 0.3879 - mse: 0.2901
Epoch 476/500
63/63 - 0s - loss: 0.3885 - mse: 0.2907
Epoch 477/500
63/63 - 0s - l

In [37]:
# 가중치 규제를 사용하지 않은 모델을 평가합니다.


R2 score: 0.586521


In [38]:
# 가중치 규제를 사용한 모델을 평가합니다.


R2 score: 0.496154


## 3. 앙상블 모델
---
금속분말 데이터셋을 이용하여 앙상블 모델을 학습해보겠습니다.


### 3.1.1 Weak 모델 정의

In [39]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
n_estimators = 10
models = []
for i in range(n_estimators):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(stage1['train_y'].shape[1])
    ])
    models.append(model)

### 3.2.2 모델 컴파일

In [40]:
for model in models:
    model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['mse'])

### 3.2.3 모델 학습

In [41]:
len_training_data = len(stage1['train_X'])
len_subset = int(len_training_data * 0.25)
histories =[]
idxs = np.arange(len_training_data)
for model in models:
    train_X = stage1['train_X'][idxs][:len_subset]
    train_y = stage1['train_y'][idxs][:len_subset]
    history = model.fit(train_X, train_y, epochs = 50, batch_size = 16, verbose = 2)
    histories.append(history)
    np.random.shuffle(idxs)

Epoch 1/50
141/141 - 0s - loss: 1.0482 - mse: 1.0482
Epoch 2/50
141/141 - 0s - loss: 0.9003 - mse: 0.9003
Epoch 3/50
141/141 - 0s - loss: 0.8483 - mse: 0.8483
Epoch 4/50
141/141 - 0s - loss: 0.8113 - mse: 0.8113
Epoch 5/50
141/141 - 0s - loss: 0.7849 - mse: 0.7849
Epoch 6/50
141/141 - 0s - loss: 0.7615 - mse: 0.7615
Epoch 7/50
141/141 - 0s - loss: 0.7390 - mse: 0.7390
Epoch 8/50
141/141 - 0s - loss: 0.7206 - mse: 0.7206
Epoch 9/50
141/141 - 0s - loss: 0.7046 - mse: 0.7046
Epoch 10/50
141/141 - 0s - loss: 0.6875 - mse: 0.6875
Epoch 11/50
141/141 - 0s - loss: 0.6751 - mse: 0.6751
Epoch 12/50
141/141 - 0s - loss: 0.6672 - mse: 0.6672
Epoch 13/50
141/141 - 0s - loss: 0.6519 - mse: 0.6519
Epoch 14/50
141/141 - 0s - loss: 0.6440 - mse: 0.6440
Epoch 15/50
141/141 - 0s - loss: 0.6325 - mse: 0.6325
Epoch 16/50
141/141 - 0s - loss: 0.6244 - mse: 0.6244
Epoch 17/50
141/141 - 0s - loss: 0.6102 - mse: 0.6102
Epoch 18/50
141/141 - 0s - loss: 0.6074 - mse: 0.6074
Epoch 19/50
141/141 - 0s - loss: 0.59

Epoch 4/50
141/141 - 0s - loss: 0.7158 - mse: 0.7158
Epoch 5/50
141/141 - 0s - loss: 0.6927 - mse: 0.6927
Epoch 6/50
141/141 - 0s - loss: 0.6761 - mse: 0.6761
Epoch 7/50
141/141 - 0s - loss: 0.6588 - mse: 0.6588
Epoch 8/50
141/141 - 0s - loss: 0.6439 - mse: 0.6439
Epoch 9/50
141/141 - 0s - loss: 0.6313 - mse: 0.6313
Epoch 10/50
141/141 - 0s - loss: 0.6216 - mse: 0.6216
Epoch 11/50
141/141 - 0s - loss: 0.6102 - mse: 0.6102
Epoch 12/50
141/141 - 0s - loss: 0.6008 - mse: 0.6008
Epoch 13/50
141/141 - 0s - loss: 0.5876 - mse: 0.5876
Epoch 14/50
141/141 - 0s - loss: 0.5774 - mse: 0.5774
Epoch 15/50
141/141 - 0s - loss: 0.5667 - mse: 0.5667
Epoch 16/50
141/141 - 0s - loss: 0.5564 - mse: 0.5564
Epoch 17/50
141/141 - 0s - loss: 0.5430 - mse: 0.5430
Epoch 18/50
141/141 - 0s - loss: 0.5362 - mse: 0.5362
Epoch 19/50
141/141 - 0s - loss: 0.5231 - mse: 0.5231
Epoch 20/50
141/141 - 0s - loss: 0.5115 - mse: 0.5115
Epoch 21/50
141/141 - 0s - loss: 0.5012 - mse: 0.5012
Epoch 22/50
141/141 - 0s - loss: 0

Epoch 7/50
141/141 - 0s - loss: 0.7670 - mse: 0.7670
Epoch 8/50
141/141 - 0s - loss: 0.7494 - mse: 0.7494
Epoch 9/50
141/141 - 0s - loss: 0.7313 - mse: 0.7313
Epoch 10/50
141/141 - 0s - loss: 0.7139 - mse: 0.7139
Epoch 11/50
141/141 - 0s - loss: 0.6977 - mse: 0.6977
Epoch 12/50
141/141 - 0s - loss: 0.6818 - mse: 0.6818
Epoch 13/50
141/141 - 0s - loss: 0.6657 - mse: 0.6657
Epoch 14/50
141/141 - 0s - loss: 0.6532 - mse: 0.6532
Epoch 15/50
141/141 - 0s - loss: 0.6445 - mse: 0.6445
Epoch 16/50
141/141 - 0s - loss: 0.6310 - mse: 0.6310
Epoch 17/50
141/141 - 0s - loss: 0.6198 - mse: 0.6198
Epoch 18/50
141/141 - 0s - loss: 0.6057 - mse: 0.6057
Epoch 19/50
141/141 - 0s - loss: 0.5963 - mse: 0.5963
Epoch 20/50
141/141 - 0s - loss: 0.5861 - mse: 0.5861
Epoch 21/50
141/141 - 0s - loss: 0.5771 - mse: 0.5771
Epoch 22/50
141/141 - 0s - loss: 0.5694 - mse: 0.5694
Epoch 23/50
141/141 - 0s - loss: 0.5631 - mse: 0.5631
Epoch 24/50
141/141 - 0s - loss: 0.5517 - mse: 0.5517
Epoch 25/50
141/141 - 0s - loss

Epoch 10/50
141/141 - 0s - loss: 0.6138 - mse: 0.6138
Epoch 11/50
141/141 - 0s - loss: 0.6002 - mse: 0.6002
Epoch 12/50
141/141 - 0s - loss: 0.5925 - mse: 0.5925
Epoch 13/50
141/141 - 0s - loss: 0.5808 - mse: 0.5808
Epoch 14/50
141/141 - 0s - loss: 0.5713 - mse: 0.5713
Epoch 15/50
141/141 - 0s - loss: 0.5659 - mse: 0.5659
Epoch 16/50
141/141 - 0s - loss: 0.5583 - mse: 0.5583
Epoch 17/50
141/141 - 0s - loss: 0.5490 - mse: 0.5490
Epoch 18/50
141/141 - 0s - loss: 0.5427 - mse: 0.5427
Epoch 19/50
141/141 - 0s - loss: 0.5335 - mse: 0.5335
Epoch 20/50
141/141 - 0s - loss: 0.5273 - mse: 0.5273
Epoch 21/50
141/141 - 0s - loss: 0.5229 - mse: 0.5229
Epoch 22/50
141/141 - 0s - loss: 0.5140 - mse: 0.5140
Epoch 23/50
141/141 - 0s - loss: 0.5069 - mse: 0.5069
Epoch 24/50
141/141 - 0s - loss: 0.5012 - mse: 0.5012
Epoch 25/50
141/141 - 0s - loss: 0.4933 - mse: 0.4933
Epoch 26/50
141/141 - 0s - loss: 0.4899 - mse: 0.4899
Epoch 27/50
141/141 - 0s - loss: 0.4834 - mse: 0.4834
Epoch 28/50
141/141 - 0s - l

### 3.2.4 모델 예측 및 평가

In [42]:
preds = 0
for i, model in enumerate(models):
    pred = model.predict(stage1['test_X'])
    r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
    print("%d 번째 weak model - R2 score: %f"%(i+1, r2))
    preds += pred
preds /= len(models)
r2 = sklearn.metrics.r2_score(stage1['test_y'], preds)
print("앙상블 모델 R2 score: %f"%r2)

1 번째 weak model - R2 score: 0.486652
2 번째 weak model - R2 score: 0.422974
3 번째 weak model - R2 score: 0.512572
4 번째 weak model - R2 score: 0.428076
5 번째 weak model - R2 score: 0.444283
6 번째 weak model - R2 score: 0.450647
7 번째 weak model - R2 score: 0.462870
8 번째 weak model - R2 score: 0.442646
9 번째 weak model - R2 score: 0.463795
10 번째 weak model - R2 score: 0.480056
앙상블 모델 R2 score: 0.532665


앙상블 모델이 각각의 약한 모델보다 성능이 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 앙상블을 적용해봅니다.

In [43]:
# Weak 모델 정의


In [44]:
# 모델 컴파일


In [45]:
# 모델 학습


Epoch 1/50
141/141 - 0s - loss: 0.9979 - mse: 0.9979
Epoch 2/50
141/141 - 0s - loss: 0.6989 - mse: 0.6989
Epoch 3/50
141/141 - 0s - loss: 0.5997 - mse: 0.5997
Epoch 4/50
141/141 - 0s - loss: 0.5533 - mse: 0.5533
Epoch 5/50
141/141 - 0s - loss: 0.5301 - mse: 0.5301
Epoch 6/50
141/141 - 0s - loss: 0.5149 - mse: 0.5149
Epoch 7/50
141/141 - 0s - loss: 0.5025 - mse: 0.5025
Epoch 8/50
141/141 - 0s - loss: 0.4943 - mse: 0.4943
Epoch 9/50
141/141 - 0s - loss: 0.4884 - mse: 0.4884
Epoch 10/50
141/141 - 0s - loss: 0.4807 - mse: 0.4807
Epoch 11/50
141/141 - 0s - loss: 0.4749 - mse: 0.4749
Epoch 12/50
141/141 - 0s - loss: 0.4679 - mse: 0.4679
Epoch 13/50
141/141 - 0s - loss: 0.4605 - mse: 0.4605
Epoch 14/50
141/141 - 0s - loss: 0.4550 - mse: 0.4550
Epoch 15/50
141/141 - 0s - loss: 0.4517 - mse: 0.4517
Epoch 16/50
141/141 - 0s - loss: 0.4421 - mse: 0.4421
Epoch 17/50
141/141 - 0s - loss: 0.4380 - mse: 0.4380
Epoch 18/50
141/141 - 0s - loss: 0.4302 - mse: 0.4302
Epoch 19/50
141/141 - 0s - loss: 0.42

Epoch 4/50
141/141 - 0s - loss: 0.4723 - mse: 0.4723
Epoch 5/50
141/141 - 0s - loss: 0.4540 - mse: 0.4540
Epoch 6/50
141/141 - 0s - loss: 0.4395 - mse: 0.4395
Epoch 7/50
141/141 - 0s - loss: 0.4267 - mse: 0.4267
Epoch 8/50
141/141 - 0s - loss: 0.4203 - mse: 0.4203
Epoch 9/50
141/141 - 0s - loss: 0.4101 - mse: 0.4101
Epoch 10/50
141/141 - 0s - loss: 0.4023 - mse: 0.4023
Epoch 11/50
141/141 - 0s - loss: 0.3965 - mse: 0.3965
Epoch 12/50
141/141 - 0s - loss: 0.3893 - mse: 0.3893
Epoch 13/50
141/141 - 0s - loss: 0.3816 - mse: 0.3816
Epoch 14/50
141/141 - 0s - loss: 0.3766 - mse: 0.3766
Epoch 15/50
141/141 - 0s - loss: 0.3706 - mse: 0.3706
Epoch 16/50
141/141 - 0s - loss: 0.3654 - mse: 0.3654
Epoch 17/50
141/141 - 0s - loss: 0.3601 - mse: 0.3601
Epoch 18/50
141/141 - 0s - loss: 0.3558 - mse: 0.3558
Epoch 19/50
141/141 - 0s - loss: 0.3510 - mse: 0.3510
Epoch 20/50
141/141 - 0s - loss: 0.3504 - mse: 0.3504
Epoch 21/50
141/141 - 0s - loss: 0.3489 - mse: 0.3489
Epoch 22/50
141/141 - 0s - loss: 0

Epoch 7/50
141/141 - 0s - loss: 0.4371 - mse: 0.4371
Epoch 8/50
141/141 - 0s - loss: 0.4243 - mse: 0.4243
Epoch 9/50
141/141 - 0s - loss: 0.4163 - mse: 0.4163
Epoch 10/50
141/141 - 0s - loss: 0.4064 - mse: 0.4064
Epoch 11/50
141/141 - 0s - loss: 0.3996 - mse: 0.3996
Epoch 12/50
141/141 - 0s - loss: 0.3921 - mse: 0.3921
Epoch 13/50
141/141 - 0s - loss: 0.3868 - mse: 0.3868
Epoch 14/50
141/141 - 0s - loss: 0.3799 - mse: 0.3799
Epoch 15/50
141/141 - 0s - loss: 0.3742 - mse: 0.3742
Epoch 16/50
141/141 - 0s - loss: 0.3672 - mse: 0.3672
Epoch 17/50
141/141 - 0s - loss: 0.3632 - mse: 0.3632
Epoch 18/50
141/141 - 0s - loss: 0.3563 - mse: 0.3563
Epoch 19/50
141/141 - 0s - loss: 0.3504 - mse: 0.3504
Epoch 20/50
141/141 - 0s - loss: 0.3479 - mse: 0.3479
Epoch 21/50
141/141 - 0s - loss: 0.3448 - mse: 0.3448
Epoch 22/50
141/141 - 0s - loss: 0.3397 - mse: 0.3397
Epoch 23/50
141/141 - 0s - loss: 0.3370 - mse: 0.3370
Epoch 24/50
141/141 - 0s - loss: 0.3342 - mse: 0.3342
Epoch 25/50
141/141 - 0s - loss

Epoch 10/50
141/141 - 0s - loss: 0.4287 - mse: 0.4287
Epoch 11/50
141/141 - 0s - loss: 0.4215 - mse: 0.4215
Epoch 12/50
141/141 - 0s - loss: 0.4174 - mse: 0.4174
Epoch 13/50
141/141 - 0s - loss: 0.4132 - mse: 0.4132
Epoch 14/50
141/141 - 0s - loss: 0.4079 - mse: 0.4079
Epoch 15/50
141/141 - 0s - loss: 0.4032 - mse: 0.4032
Epoch 16/50
141/141 - 0s - loss: 0.4001 - mse: 0.4001
Epoch 17/50
141/141 - 0s - loss: 0.3954 - mse: 0.3954
Epoch 18/50
141/141 - 0s - loss: 0.3912 - mse: 0.3912
Epoch 19/50
141/141 - 0s - loss: 0.3888 - mse: 0.3888
Epoch 20/50
141/141 - 0s - loss: 0.3850 - mse: 0.3850
Epoch 21/50
141/141 - 0s - loss: 0.3834 - mse: 0.3834
Epoch 22/50
141/141 - 0s - loss: 0.3797 - mse: 0.3797
Epoch 23/50
141/141 - 0s - loss: 0.3754 - mse: 0.3754
Epoch 24/50
141/141 - 0s - loss: 0.3722 - mse: 0.3722
Epoch 25/50
141/141 - 0s - loss: 0.3705 - mse: 0.3705
Epoch 26/50
141/141 - 0s - loss: 0.3675 - mse: 0.3675
Epoch 27/50
141/141 - 0s - loss: 0.3645 - mse: 0.3645
Epoch 28/50
141/141 - 0s - l

In [46]:
# 모델 예측 및 평가


1 번째 weak model - R2 score: 0.605306
2 번째 weak model - R2 score: 0.619723
3 번째 weak model - R2 score: 0.619247
4 번째 weak model - R2 score: 0.586581
5 번째 weak model - R2 score: 0.603946
6 번째 weak model - R2 score: 0.632913
7 번째 weak model - R2 score: 0.645047
8 번째 weak model - R2 score: 0.626863
9 번째 weak model - R2 score: 0.636776
10 번째 weak model - R2 score: 0.576855
앙상블 모델 R2 score: 0.660411


## 4. Dropout
---
금속분말 데이터셋을 이용하여 Dropout 을 적용해보겠습니다.


### 4.1 모델 정의

In [47]:
# MLP 모델을 설정합니다.
# Dropout을 하지 않을 모델입니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# Dropout을 사용할 모델입니다. rate는 0.2로 설정합니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_dropout = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dropout(rate=0.2), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 4.2 모델 컴파일

In [48]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])
MLP_model_dropout.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])

### 4.3 모델 학습

In [49]:
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.1605 - mse: 1.1605
Epoch 2/500
63/63 - 0s - loss: 1.1044 - mse: 1.1044
Epoch 3/500
63/63 - 0s - loss: 1.0752 - mse: 1.0752
Epoch 4/500
63/63 - 0s - loss: 1.0524 - mse: 1.0524
Epoch 5/500
63/63 - 0s - loss: 1.0320 - mse: 1.0320
Epoch 6/500
63/63 - 0s - loss: 1.0123 - mse: 1.0123
Epoch 7/500
63/63 - 0s - loss: 0.9935 - mse: 0.9935
Epoch 8/500
63/63 - 0s - loss: 0.9763 - mse: 0.9763
Epoch 9/500
63/63 - 0s - loss: 0.9609 - mse: 0.9609
Epoch 10/500
63/63 - 0s - loss: 0.9462 - mse: 0.9462
Epoch 11/500
63/63 - 0s - loss: 0.9324 - mse: 0.9324
Epoch 12/500
63/63 - 0s - loss: 0.9203 - mse: 0.9203
Epoch 13/500
63/63 - 0s - loss: 0.9085 - mse: 0.9085
Epoch 14/500
63/63 - 0s - loss: 0.8980 - mse: 0.8980
Epoch 15/500
63/63 - 0s - loss: 0.8886 - mse: 0.8886
Epoch 16/500
63/63 - 0s - loss: 0.8800 - mse: 0.8800
Epoch 17/500
63/63 - 0s - loss: 0.8723 - mse: 0.8723
Epoch 18/500
63/63 - 0s - loss: 0.8655 - mse: 0.8655
Epoch 19/500
63/63 - 0s - loss: 0.8590 - mse: 0.8590
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3061 - mse: 0.3061
Epoch 156/500
63/63 - 0s - loss: 0.3051 - mse: 0.3051
Epoch 157/500
63/63 - 0s - loss: 0.3043 - mse: 0.3043
Epoch 158/500
63/63 - 0s - loss: 0.3035 - mse: 0.3035
Epoch 159/500
63/63 - 0s - loss: 0.3027 - mse: 0.3027
Epoch 160/500
63/63 - 0s - loss: 0.3022 - mse: 0.3022
Epoch 161/500
63/63 - 0s - loss: 0.3010 - mse: 0.3010
Epoch 162/500
63/63 - 0s - loss: 0.2998 - mse: 0.2998
Epoch 163/500
63/63 - 0s - loss: 0.2994 - mse: 0.2994
Epoch 164/500
63/63 - 0s - loss: 0.2981 - mse: 0.2981
Epoch 165/500
63/63 - 0s - loss: 0.2978 - mse: 0.2978
Epoch 166/500
63/63 - 0s - loss: 0.2973 - mse: 0.2973
Epoch 167/500
63/63 - 0s - loss: 0.2962 - mse: 0.2962
Epoch 168/500
63/63 - 0s - loss: 0.2952 - mse: 0.2952
Epoch 169/500
63/63 - 0s - loss: 0.2946 - mse: 0.2946
Epoch 170/500
63/63 - 0s - loss: 0.2939 - mse: 0.2939
Epoch 171/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929
Epoch 172/500
63/63 - 0s - loss: 0.2923 - mse: 0.2923
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2175 - mse: 0.2175
Epoch 308/500
63/63 - 0s - loss: 0.2168 - mse: 0.2168
Epoch 309/500
63/63 - 0s - loss: 0.2162 - mse: 0.2162
Epoch 310/500
63/63 - 0s - loss: 0.2160 - mse: 0.2160
Epoch 311/500
63/63 - 0s - loss: 0.2150 - mse: 0.2150
Epoch 312/500
63/63 - 0s - loss: 0.2147 - mse: 0.2147
Epoch 313/500
63/63 - 0s - loss: 0.2142 - mse: 0.2142
Epoch 314/500
63/63 - 0s - loss: 0.2141 - mse: 0.2141
Epoch 315/500
63/63 - 0s - loss: 0.2137 - mse: 0.2137
Epoch 316/500
63/63 - 0s - loss: 0.2130 - mse: 0.2130
Epoch 317/500
63/63 - 0s - loss: 0.2126 - mse: 0.2126
Epoch 318/500
63/63 - 0s - loss: 0.2118 - mse: 0.2118
Epoch 319/500
63/63 - 0s - loss: 0.2117 - mse: 0.2117
Epoch 320/500
63/63 - 0s - loss: 0.2108 - mse: 0.2108
Epoch 321/500
63/63 - 0s - loss: 0.2103 - mse: 0.2103
Epoch 322/500
63/63 - 0s - loss: 0.2100 - mse: 0.2100
Epoch 323/500
63/63 - 0s - loss: 0.2093 - mse: 0.2093
Epoch 324/500
63/63 - 0s - loss: 0.2090 - mse: 0.2090
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1534 - mse: 0.1534
Epoch 460/500
63/63 - 0s - loss: 0.1528 - mse: 0.1528
Epoch 461/500
63/63 - 0s - loss: 0.1527 - mse: 0.1527
Epoch 462/500
63/63 - 0s - loss: 0.1527 - mse: 0.1527
Epoch 463/500
63/63 - 0s - loss: 0.1519 - mse: 0.1519
Epoch 464/500
63/63 - 0s - loss: 0.1513 - mse: 0.1513
Epoch 465/500
63/63 - 0s - loss: 0.1511 - mse: 0.1511
Epoch 466/500
63/63 - 0s - loss: 0.1508 - mse: 0.1508
Epoch 467/500
63/63 - 0s - loss: 0.1507 - mse: 0.1507
Epoch 468/500
63/63 - 0s - loss: 0.1501 - mse: 0.1501
Epoch 469/500
63/63 - 0s - loss: 0.1503 - mse: 0.1503
Epoch 470/500
63/63 - 0s - loss: 0.1498 - mse: 0.1498
Epoch 471/500
63/63 - 0s - loss: 0.1491 - mse: 0.1491
Epoch 472/500
63/63 - 0s - loss: 0.1490 - mse: 0.1490
Epoch 473/500
63/63 - 0s - loss: 0.1490 - mse: 0.1490
Epoch 474/500
63/63 - 0s - loss: 0.1481 - mse: 0.1481
Epoch 475/500
63/63 - 0s - loss: 0.1481 - mse: 0.1481
Epoch 476/500
63/63 - 0s - loss: 0.1474 - mse: 0.1474
Epoch 477/500
63/63 - 0s - l

In [50]:
history = MLP_model_dropout.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.2291 - mse: 1.2291
Epoch 2/500
63/63 - 0s - loss: 1.1445 - mse: 1.1445
Epoch 3/500
63/63 - 0s - loss: 1.1226 - mse: 1.1226
Epoch 4/500
63/63 - 0s - loss: 1.1013 - mse: 1.1013
Epoch 5/500
63/63 - 0s - loss: 1.0819 - mse: 1.0819
Epoch 6/500
63/63 - 0s - loss: 1.0703 - mse: 1.0703
Epoch 7/500
63/63 - 0s - loss: 1.0512 - mse: 1.0512
Epoch 8/500
63/63 - 0s - loss: 1.0386 - mse: 1.0386
Epoch 9/500
63/63 - 0s - loss: 1.0252 - mse: 1.0252
Epoch 10/500
63/63 - 0s - loss: 1.0155 - mse: 1.0155
Epoch 11/500
63/63 - 0s - loss: 1.0028 - mse: 1.0028
Epoch 12/500
63/63 - 0s - loss: 1.0028 - mse: 1.0028
Epoch 13/500
63/63 - 0s - loss: 0.9886 - mse: 0.9886
Epoch 14/500
63/63 - 0s - loss: 0.9845 - mse: 0.9845
Epoch 15/500
63/63 - 0s - loss: 0.9846 - mse: 0.9846
Epoch 16/500
63/63 - 0s - loss: 0.9745 - mse: 0.9745
Epoch 17/500
63/63 - 0s - loss: 0.9685 - mse: 0.9685
Epoch 18/500
63/63 - 0s - loss: 0.9639 - mse: 0.9639
Epoch 19/500
63/63 - 0s - loss: 0.9591 - mse: 0.9591
Ep

Epoch 155/500
63/63 - 0s - loss: 0.5716 - mse: 0.5716
Epoch 156/500
63/63 - 0s - loss: 0.4998 - mse: 0.4998
Epoch 157/500
63/63 - 0s - loss: 0.4782 - mse: 0.4782
Epoch 158/500
63/63 - 0s - loss: 0.4989 - mse: 0.4989
Epoch 159/500
63/63 - 0s - loss: 0.5462 - mse: 0.5462
Epoch 160/500
63/63 - 0s - loss: 0.5096 - mse: 0.5096
Epoch 161/500
63/63 - 0s - loss: 0.5217 - mse: 0.5217
Epoch 162/500
63/63 - 0s - loss: 0.5228 - mse: 0.5228
Epoch 163/500
63/63 - 0s - loss: 0.5217 - mse: 0.5217
Epoch 164/500
63/63 - 0s - loss: 0.5394 - mse: 0.5394
Epoch 165/500
63/63 - 0s - loss: 0.5463 - mse: 0.5463
Epoch 166/500
63/63 - 0s - loss: 0.4952 - mse: 0.4952
Epoch 167/500
63/63 - 0s - loss: 0.5162 - mse: 0.5162
Epoch 168/500
63/63 - 0s - loss: 0.4803 - mse: 0.4803
Epoch 169/500
63/63 - 0s - loss: 0.5361 - mse: 0.5361
Epoch 170/500
63/63 - 0s - loss: 0.5566 - mse: 0.5566
Epoch 171/500
63/63 - 0s - loss: 0.5383 - mse: 0.5383
Epoch 172/500
63/63 - 0s - loss: 0.5209 - mse: 0.5209
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.4441 - mse: 0.4441
Epoch 308/500
63/63 - 0s - loss: 0.4289 - mse: 0.4289
Epoch 309/500
63/63 - 0s - loss: 0.4097 - mse: 0.4097
Epoch 310/500
63/63 - 0s - loss: 0.4072 - mse: 0.4072
Epoch 311/500
63/63 - 0s - loss: 0.4286 - mse: 0.4286
Epoch 312/500
63/63 - 0s - loss: 0.5041 - mse: 0.5041
Epoch 313/500
63/63 - 0s - loss: 0.4337 - mse: 0.4337
Epoch 314/500
63/63 - 0s - loss: 0.4320 - mse: 0.4320
Epoch 315/500
63/63 - 0s - loss: 0.4570 - mse: 0.4570
Epoch 316/500
63/63 - 0s - loss: 0.4148 - mse: 0.4148
Epoch 317/500
63/63 - 0s - loss: 0.4178 - mse: 0.4178
Epoch 318/500
63/63 - 0s - loss: 0.4047 - mse: 0.4047
Epoch 319/500
63/63 - 0s - loss: 0.4220 - mse: 0.4220
Epoch 320/500
63/63 - 0s - loss: 0.4570 - mse: 0.4570
Epoch 321/500
63/63 - 0s - loss: 0.4367 - mse: 0.4367
Epoch 322/500
63/63 - 0s - loss: 0.4271 - mse: 0.4271
Epoch 323/500
63/63 - 0s - loss: 0.4207 - mse: 0.4207
Epoch 324/500
63/63 - 0s - loss: 0.4165 - mse: 0.4165
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.3955 - mse: 0.3955
Epoch 460/500
63/63 - 0s - loss: 0.3960 - mse: 0.3960
Epoch 461/500
63/63 - 0s - loss: 0.3958 - mse: 0.3958
Epoch 462/500
63/63 - 0s - loss: 0.4080 - mse: 0.4080
Epoch 463/500
63/63 - 0s - loss: 0.4225 - mse: 0.4225
Epoch 464/500
63/63 - 0s - loss: 0.4395 - mse: 0.4395
Epoch 465/500
63/63 - 0s - loss: 0.3930 - mse: 0.3930
Epoch 466/500
63/63 - 0s - loss: 0.4132 - mse: 0.4132
Epoch 467/500
63/63 - 0s - loss: 0.3727 - mse: 0.3727
Epoch 468/500
63/63 - 0s - loss: 0.3901 - mse: 0.3901
Epoch 469/500
63/63 - 0s - loss: 0.4151 - mse: 0.4151
Epoch 470/500
63/63 - 0s - loss: 0.3912 - mse: 0.3912
Epoch 471/500
63/63 - 0s - loss: 0.4199 - mse: 0.4199
Epoch 472/500
63/63 - 0s - loss: 0.3877 - mse: 0.3877
Epoch 473/500
63/63 - 0s - loss: 0.4121 - mse: 0.4121
Epoch 474/500
63/63 - 0s - loss: 0.3977 - mse: 0.3977
Epoch 475/500
63/63 - 0s - loss: 0.4145 - mse: 0.4145
Epoch 476/500
63/63 - 0s - loss: 0.3894 - mse: 0.3894
Epoch 477/500
63/63 - 0s - l

### 4.3 모델 예측 및 평가

In [51]:
pred = MLP_model.predict(stage1['test_X'])

In [52]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.293131


In [53]:
pred = MLP_model_dropout.predict(stage1['test_X'])

In [54]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.385034


Dropout 을 사용한 모델이 더 성능이 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 Dropout을 적용해봅니다.

In [55]:
# MLP 모델을 설정합니다. Dropout 을 사용할 모델과 사용하지 않을 모델을 모두 정의합니다.


In [56]:
# 모델 컴파일


In [57]:
# Dropout 을 사용하지 않을 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0007 - mse: 1.0007
Epoch 2/500
63/63 - 0s - loss: 0.8786 - mse: 0.8786
Epoch 3/500
63/63 - 0s - loss: 0.7753 - mse: 0.7753
Epoch 4/500
63/63 - 0s - loss: 0.6962 - mse: 0.6962
Epoch 5/500
63/63 - 0s - loss: 0.6453 - mse: 0.6453
Epoch 6/500
63/63 - 0s - loss: 0.6155 - mse: 0.6155
Epoch 7/500
63/63 - 0s - loss: 0.5943 - mse: 0.5943
Epoch 8/500
63/63 - 0s - loss: 0.5782 - mse: 0.5782
Epoch 9/500
63/63 - 0s - loss: 0.5644 - mse: 0.5644
Epoch 10/500
63/63 - 0s - loss: 0.5515 - mse: 0.5515
Epoch 11/500
63/63 - 0s - loss: 0.5415 - mse: 0.5415
Epoch 12/500
63/63 - 0s - loss: 0.5322 - mse: 0.5322
Epoch 13/500
63/63 - 0s - loss: 0.5234 - mse: 0.5234
Epoch 14/500
63/63 - 0s - loss: 0.5148 - mse: 0.5148
Epoch 15/500
63/63 - 0s - loss: 0.5086 - mse: 0.5086
Epoch 16/500
63/63 - 0s - loss: 0.5013 - mse: 0.5013
Epoch 17/500
63/63 - 0s - loss: 0.4955 - mse: 0.4955
Epoch 18/500
63/63 - 0s - loss: 0.4904 - mse: 0.4904
Epoch 19/500
63/63 - 0s - loss: 0.4858 - mse: 0.4858
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3337 - mse: 0.3337
Epoch 156/500
63/63 - 0s - loss: 0.3329 - mse: 0.3329
Epoch 157/500
63/63 - 0s - loss: 0.3331 - mse: 0.3331
Epoch 158/500
63/63 - 0s - loss: 0.3326 - mse: 0.3326
Epoch 159/500
63/63 - 0s - loss: 0.3316 - mse: 0.3316
Epoch 160/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 161/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 162/500
63/63 - 0s - loss: 0.3300 - mse: 0.3300
Epoch 163/500
63/63 - 0s - loss: 0.3292 - mse: 0.3292
Epoch 164/500
63/63 - 0s - loss: 0.3286 - mse: 0.3286
Epoch 165/500
63/63 - 0s - loss: 0.3283 - mse: 0.3283
Epoch 166/500
63/63 - 0s - loss: 0.3277 - mse: 0.3277
Epoch 167/500
63/63 - 0s - loss: 0.3276 - mse: 0.3276
Epoch 168/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 169/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 170/500
63/63 - 0s - loss: 0.3250 - mse: 0.3250
Epoch 171/500
63/63 - 0s - loss: 0.3262 - mse: 0.3262
Epoch 172/500
63/63 - 0s - loss: 0.3232 - mse: 0.3232
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2814 - mse: 0.2814
Epoch 308/500
63/63 - 0s - loss: 0.2809 - mse: 0.2809
Epoch 309/500
63/63 - 0s - loss: 0.2813 - mse: 0.2813
Epoch 310/500
63/63 - 0s - loss: 0.2804 - mse: 0.2804
Epoch 311/500
63/63 - 0s - loss: 0.2799 - mse: 0.2799
Epoch 312/500
63/63 - 0s - loss: 0.2795 - mse: 0.2795
Epoch 313/500
63/63 - 0s - loss: 0.2799 - mse: 0.2799
Epoch 314/500
63/63 - 0s - loss: 0.2797 - mse: 0.2797
Epoch 315/500
63/63 - 0s - loss: 0.2787 - mse: 0.2787
Epoch 316/500
63/63 - 0s - loss: 0.2789 - mse: 0.2789
Epoch 317/500
63/63 - 0s - loss: 0.2787 - mse: 0.2787
Epoch 318/500
63/63 - 0s - loss: 0.2775 - mse: 0.2775
Epoch 319/500
63/63 - 0s - loss: 0.2789 - mse: 0.2789
Epoch 320/500
63/63 - 0s - loss: 0.2782 - mse: 0.2782
Epoch 321/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 322/500
63/63 - 0s - loss: 0.2786 - mse: 0.2786
Epoch 323/500
63/63 - 0s - loss: 0.2786 - mse: 0.2786
Epoch 324/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2525 - mse: 0.2525
Epoch 460/500
63/63 - 0s - loss: 0.2514 - mse: 0.2514
Epoch 461/500
63/63 - 0s - loss: 0.2520 - mse: 0.2520
Epoch 462/500
63/63 - 0s - loss: 0.2533 - mse: 0.2533
Epoch 463/500
63/63 - 0s - loss: 0.2524 - mse: 0.2524
Epoch 464/500
63/63 - 0s - loss: 0.2511 - mse: 0.2511
Epoch 465/500
63/63 - 0s - loss: 0.2513 - mse: 0.2513
Epoch 466/500
63/63 - 0s - loss: 0.2517 - mse: 0.2517
Epoch 467/500
63/63 - 0s - loss: 0.2505 - mse: 0.2505
Epoch 468/500
63/63 - 0s - loss: 0.2506 - mse: 0.2506
Epoch 469/500
63/63 - 0s - loss: 0.2502 - mse: 0.2502
Epoch 470/500
63/63 - 0s - loss: 0.2511 - mse: 0.2511
Epoch 471/500
63/63 - 0s - loss: 0.2505 - mse: 0.2505
Epoch 472/500
63/63 - 0s - loss: 0.2500 - mse: 0.2500
Epoch 473/500
63/63 - 0s - loss: 0.2499 - mse: 0.2499
Epoch 474/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 475/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 476/500
63/63 - 0s - loss: 0.2495 - mse: 0.2495
Epoch 477/500
63/63 - 0s - l

In [58]:
# Dropout 을 사용할 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0805 - mse: 1.0805
Epoch 2/500
63/63 - 0s - loss: 0.9731 - mse: 0.9731
Epoch 3/500
63/63 - 0s - loss: 0.9207 - mse: 0.9207
Epoch 4/500
63/63 - 0s - loss: 0.8152 - mse: 0.8152
Epoch 5/500
63/63 - 0s - loss: 0.7873 - mse: 0.7873
Epoch 6/500
63/63 - 0s - loss: 0.7973 - mse: 0.7973
Epoch 7/500
63/63 - 0s - loss: 0.7524 - mse: 0.7524
Epoch 8/500
63/63 - 0s - loss: 0.7176 - mse: 0.7176
Epoch 9/500
63/63 - 0s - loss: 0.7376 - mse: 0.7376
Epoch 10/500
63/63 - 0s - loss: 0.7067 - mse: 0.7067
Epoch 11/500
63/63 - 0s - loss: 0.6997 - mse: 0.6997
Epoch 12/500
63/63 - 0s - loss: 0.6948 - mse: 0.6948
Epoch 13/500
63/63 - 0s - loss: 0.6834 - mse: 0.6834
Epoch 14/500
63/63 - 0s - loss: 0.6716 - mse: 0.6716
Epoch 15/500
63/63 - 0s - loss: 0.6849 - mse: 0.6849
Epoch 16/500
63/63 - 0s - loss: 0.6826 - mse: 0.6826
Epoch 17/500
63/63 - 0s - loss: 0.6695 - mse: 0.6695
Epoch 18/500
63/63 - 0s - loss: 0.6403 - mse: 0.6403
Epoch 19/500
63/63 - 0s - loss: 0.6305 - mse: 0.6305
Ep

Epoch 155/500
63/63 - 0s - loss: 0.4724 - mse: 0.4724
Epoch 156/500
63/63 - 0s - loss: 0.4609 - mse: 0.4609
Epoch 157/500
63/63 - 0s - loss: 0.4771 - mse: 0.4771
Epoch 158/500
63/63 - 0s - loss: 0.4950 - mse: 0.4950
Epoch 159/500
63/63 - 0s - loss: 0.4693 - mse: 0.4693
Epoch 160/500
63/63 - 0s - loss: 0.4871 - mse: 0.4871
Epoch 161/500
63/63 - 0s - loss: 0.4787 - mse: 0.4787
Epoch 162/500
63/63 - 0s - loss: 0.4812 - mse: 0.4812
Epoch 163/500
63/63 - 0s - loss: 0.4869 - mse: 0.4869
Epoch 164/500
63/63 - 0s - loss: 0.4678 - mse: 0.4678
Epoch 165/500
63/63 - 0s - loss: 0.4881 - mse: 0.4881
Epoch 166/500
63/63 - 0s - loss: 0.4665 - mse: 0.4665
Epoch 167/500
63/63 - 0s - loss: 0.4672 - mse: 0.4672
Epoch 168/500
63/63 - 0s - loss: 0.4728 - mse: 0.4728
Epoch 169/500
63/63 - 0s - loss: 0.4582 - mse: 0.4582
Epoch 170/500
63/63 - 0s - loss: 0.4785 - mse: 0.4785
Epoch 171/500
63/63 - 0s - loss: 0.4754 - mse: 0.4754
Epoch 172/500
63/63 - 0s - loss: 0.4606 - mse: 0.4606
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.4210 - mse: 0.4210
Epoch 308/500
63/63 - 0s - loss: 0.4318 - mse: 0.4318
Epoch 309/500
63/63 - 0s - loss: 0.4341 - mse: 0.4341
Epoch 310/500
63/63 - 0s - loss: 0.4258 - mse: 0.4258
Epoch 311/500
63/63 - 0s - loss: 0.4357 - mse: 0.4357
Epoch 312/500
63/63 - 0s - loss: 0.4181 - mse: 0.4181
Epoch 313/500
63/63 - 0s - loss: 0.4263 - mse: 0.4263
Epoch 314/500
63/63 - 0s - loss: 0.4244 - mse: 0.4244
Epoch 315/500
63/63 - 0s - loss: 0.4027 - mse: 0.4027
Epoch 316/500
63/63 - 0s - loss: 0.4164 - mse: 0.4164
Epoch 317/500
63/63 - 0s - loss: 0.4303 - mse: 0.4303
Epoch 318/500
63/63 - 0s - loss: 0.4144 - mse: 0.4144
Epoch 319/500
63/63 - 0s - loss: 0.4256 - mse: 0.4256
Epoch 320/500
63/63 - 0s - loss: 0.4163 - mse: 0.4163
Epoch 321/500
63/63 - 0s - loss: 0.4431 - mse: 0.4431
Epoch 322/500
63/63 - 0s - loss: 0.4057 - mse: 0.4057
Epoch 323/500
63/63 - 0s - loss: 0.4112 - mse: 0.4112
Epoch 324/500
63/63 - 0s - loss: 0.4175 - mse: 0.4175
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.3842 - mse: 0.3842
Epoch 460/500
63/63 - 0s - loss: 0.3809 - mse: 0.3809
Epoch 461/500
63/63 - 0s - loss: 0.4130 - mse: 0.4130
Epoch 462/500
63/63 - 0s - loss: 0.3807 - mse: 0.3807
Epoch 463/500
63/63 - 0s - loss: 0.3820 - mse: 0.3820
Epoch 464/500
63/63 - 0s - loss: 0.3768 - mse: 0.3768
Epoch 465/500
63/63 - 0s - loss: 0.3881 - mse: 0.3881
Epoch 466/500
63/63 - 0s - loss: 0.3919 - mse: 0.3919
Epoch 467/500
63/63 - 0s - loss: 0.3806 - mse: 0.3806
Epoch 468/500
63/63 - 0s - loss: 0.4077 - mse: 0.4077
Epoch 469/500
63/63 - 0s - loss: 0.3728 - mse: 0.3728
Epoch 470/500
63/63 - 0s - loss: 0.3891 - mse: 0.3891
Epoch 471/500
63/63 - 0s - loss: 0.3908 - mse: 0.3908
Epoch 472/500
63/63 - 0s - loss: 0.3880 - mse: 0.3880
Epoch 473/500
63/63 - 0s - loss: 0.3802 - mse: 0.3802
Epoch 474/500
63/63 - 0s - loss: 0.3971 - mse: 0.3971
Epoch 475/500
63/63 - 0s - loss: 0.3924 - mse: 0.3924
Epoch 476/500
63/63 - 0s - loss: 0.3951 - mse: 0.3951
Epoch 477/500
63/63 - 0s - l

In [59]:
# Dropout 을 사용하지 않은 모델을 평가합니다.


R2 score: 0.588310


In [60]:
# Dropout 을 사용한 않은 모델을 평가합니다.


R2 score: 0.599816


## 5. 정규화
---
금속분말 데이터셋을 이용하여 정규화를 수행해보겠습니다.

### 5.1 모델 정의

In [61]:
# MLP 모델을 설정합니다.
# 정규화를 하지 않을 모델입니다. Dropout에서 학습한 모델과 같은 구조를 사용하겠습니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 정규화를 사용할 모델입니다. Batch normalization 을 사용해보겠습니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_ln = tf.keras.Sequential([
    tf.keras.layers.Dense(128), 
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 5.2 모델 컴파일

In [62]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['mse'])
MLP_model_ln.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['mse'])

### 5.3 모델 학습

In [63]:
# 정규화를 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.0473 - mse: 1.0473
Epoch 2/500
63/63 - 0s - loss: 0.9160 - mse: 0.9160
Epoch 3/500
63/63 - 0s - loss: 0.8524 - mse: 0.8524
Epoch 4/500
63/63 - 0s - loss: 0.8108 - mse: 0.8108
Epoch 5/500
63/63 - 0s - loss: 0.7759 - mse: 0.7759
Epoch 6/500
63/63 - 0s - loss: 0.7346 - mse: 0.7346
Epoch 7/500
63/63 - 0s - loss: 0.6965 - mse: 0.6965
Epoch 8/500
63/63 - 0s - loss: 0.6596 - mse: 0.6596
Epoch 9/500
63/63 - 0s - loss: 0.6215 - mse: 0.6215
Epoch 10/500
63/63 - 0s - loss: 0.5943 - mse: 0.5943
Epoch 11/500
63/63 - 0s - loss: 0.5789 - mse: 0.5789
Epoch 12/500
63/63 - 0s - loss: 0.5300 - mse: 0.5300
Epoch 13/500
63/63 - 0s - loss: 0.4923 - mse: 0.4923
Epoch 14/500
63/63 - 0s - loss: 0.4805 - mse: 0.4805
Epoch 15/500
63/63 - 0s - loss: 0.4549 - mse: 0.4549
Epoch 16/500
63/63 - 0s - loss: 0.4176 - mse: 0.4176
Epoch 17/500
63/63 - 0s - loss: 0.3719 - mse: 0.3719
Epoch 18/500
63/63 - 0s - loss: 0.3607 - mse: 0.3607
Epoch 19/500
63/63 - 0s - loss: 0.3362 - mse: 0.3362
Ep

Epoch 155/500
63/63 - 0s - loss: 0.0282 - mse: 0.0282
Epoch 156/500
63/63 - 0s - loss: 0.0278 - mse: 0.0278
Epoch 157/500
63/63 - 0s - loss: 0.0280 - mse: 0.0280
Epoch 158/500
63/63 - 0s - loss: 0.0270 - mse: 0.0270
Epoch 159/500
63/63 - 0s - loss: 0.0280 - mse: 0.0280
Epoch 160/500
63/63 - 0s - loss: 0.0295 - mse: 0.0295
Epoch 161/500
63/63 - 0s - loss: 0.0327 - mse: 0.0327
Epoch 162/500
63/63 - 0s - loss: 0.0317 - mse: 0.0317
Epoch 163/500
63/63 - 0s - loss: 0.0307 - mse: 0.0307
Epoch 164/500
63/63 - 0s - loss: 0.0296 - mse: 0.0296
Epoch 165/500
63/63 - 0s - loss: 0.0300 - mse: 0.0300
Epoch 166/500
63/63 - 0s - loss: 0.0369 - mse: 0.0369
Epoch 167/500
63/63 - 0s - loss: 0.0378 - mse: 0.0378
Epoch 168/500
63/63 - 0s - loss: 0.0355 - mse: 0.0355
Epoch 169/500
63/63 - 0s - loss: 0.0433 - mse: 0.0433
Epoch 170/500
63/63 - 0s - loss: 0.0738 - mse: 0.0738
Epoch 171/500
63/63 - 0s - loss: 0.0586 - mse: 0.0586
Epoch 172/500
63/63 - 0s - loss: 0.0480 - mse: 0.0480
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.0119 - mse: 0.0119
Epoch 308/500
63/63 - 0s - loss: 0.0147 - mse: 0.0147
Epoch 309/500
63/63 - 0s - loss: 0.0156 - mse: 0.0156
Epoch 310/500
63/63 - 0s - loss: 0.0165 - mse: 0.0165
Epoch 311/500
63/63 - 0s - loss: 0.0183 - mse: 0.0183
Epoch 312/500
63/63 - 0s - loss: 0.0218 - mse: 0.0218
Epoch 313/500
63/63 - 0s - loss: 0.0327 - mse: 0.0327
Epoch 314/500
63/63 - 0s - loss: 0.0293 - mse: 0.0293
Epoch 315/500
63/63 - 0s - loss: 0.0201 - mse: 0.0201
Epoch 316/500
63/63 - 0s - loss: 0.0218 - mse: 0.0218
Epoch 317/500
63/63 - 0s - loss: 0.0206 - mse: 0.0206
Epoch 318/500
63/63 - 0s - loss: 0.0176 - mse: 0.0176
Epoch 319/500
63/63 - 0s - loss: 0.0147 - mse: 0.0147
Epoch 320/500
63/63 - 0s - loss: 0.0120 - mse: 0.0120
Epoch 321/500
63/63 - 0s - loss: 0.0114 - mse: 0.0114
Epoch 322/500
63/63 - 0s - loss: 0.0108 - mse: 0.0108
Epoch 323/500
63/63 - 0s - loss: 0.0122 - mse: 0.0122
Epoch 324/500
63/63 - 0s - loss: 0.0117 - mse: 0.0117
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.0111 - mse: 0.0111
Epoch 460/500
63/63 - 0s - loss: 0.0088 - mse: 0.0088
Epoch 461/500
63/63 - 0s - loss: 0.0077 - mse: 0.0077
Epoch 462/500
63/63 - 0s - loss: 0.0074 - mse: 0.0074
Epoch 463/500
63/63 - 0s - loss: 0.0070 - mse: 0.0070
Epoch 464/500
63/63 - 0s - loss: 0.0068 - mse: 0.0068
Epoch 465/500
63/63 - 0s - loss: 0.0063 - mse: 0.0063
Epoch 466/500
63/63 - 0s - loss: 0.0061 - mse: 0.0061
Epoch 467/500
63/63 - 0s - loss: 0.0064 - mse: 0.0064
Epoch 468/500
63/63 - 0s - loss: 0.0067 - mse: 0.0067
Epoch 469/500
63/63 - 0s - loss: 0.0076 - mse: 0.0076
Epoch 470/500
63/63 - 0s - loss: 0.0089 - mse: 0.0089
Epoch 471/500
63/63 - 0s - loss: 0.0091 - mse: 0.0091
Epoch 472/500
63/63 - 0s - loss: 0.0111 - mse: 0.0111
Epoch 473/500
63/63 - 0s - loss: 0.0117 - mse: 0.0117
Epoch 474/500
63/63 - 0s - loss: 0.0125 - mse: 0.0125
Epoch 475/500
63/63 - 0s - loss: 0.0173 - mse: 0.0173
Epoch 476/500
63/63 - 0s - loss: 0.0175 - mse: 0.0175
Epoch 477/500
63/63 - 0s - l

In [64]:
# 정규화를 사용할 모델을 학습합니다.
history = MLP_model_ln.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 1s - loss: 1.2281 - mse: 1.2281
Epoch 2/500
63/63 - 0s - loss: 0.9434 - mse: 0.9434
Epoch 3/500
63/63 - 0s - loss: 0.8920 - mse: 0.8920
Epoch 4/500
63/63 - 0s - loss: 0.8610 - mse: 0.8610
Epoch 5/500
63/63 - 0s - loss: 0.8287 - mse: 0.8287
Epoch 6/500
63/63 - 0s - loss: 0.8144 - mse: 0.8144
Epoch 7/500
63/63 - 0s - loss: 0.7945 - mse: 0.7945
Epoch 8/500
63/63 - 0s - loss: 0.7799 - mse: 0.7799
Epoch 9/500
63/63 - 0s - loss: 0.7715 - mse: 0.7715
Epoch 10/500
63/63 - 0s - loss: 0.7479 - mse: 0.7479
Epoch 11/500
63/63 - 0s - loss: 0.7453 - mse: 0.7453
Epoch 12/500
63/63 - 0s - loss: 0.7381 - mse: 0.7381
Epoch 13/500
63/63 - 0s - loss: 0.7206 - mse: 0.7206
Epoch 14/500
63/63 - 0s - loss: 0.7001 - mse: 0.7001
Epoch 15/500
63/63 - 0s - loss: 0.7009 - mse: 0.7009
Epoch 16/500
63/63 - 0s - loss: 0.6984 - mse: 0.6984
Epoch 17/500
63/63 - 0s - loss: 0.6817 - mse: 0.6817
Epoch 18/500
63/63 - 0s - loss: 0.6546 - mse: 0.6546
Epoch 19/500
63/63 - 0s - loss: 0.6668 - mse: 0.6668
Ep

Epoch 155/500
63/63 - 0s - loss: 0.2542 - mse: 0.2542
Epoch 156/500
63/63 - 0s - loss: 0.2578 - mse: 0.2578
Epoch 157/500
63/63 - 0s - loss: 0.2753 - mse: 0.2753
Epoch 158/500
63/63 - 0s - loss: 0.2573 - mse: 0.2573
Epoch 159/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 160/500
63/63 - 0s - loss: 0.2516 - mse: 0.2516
Epoch 161/500
63/63 - 0s - loss: 0.2545 - mse: 0.2545
Epoch 162/500
63/63 - 0s - loss: 0.2542 - mse: 0.2542
Epoch 163/500
63/63 - 0s - loss: 0.2592 - mse: 0.2592
Epoch 164/500
63/63 - 0s - loss: 0.2634 - mse: 0.2634
Epoch 165/500
63/63 - 0s - loss: 0.2715 - mse: 0.2715
Epoch 166/500
63/63 - 0s - loss: 0.2716 - mse: 0.2716
Epoch 167/500
63/63 - 0s - loss: 0.2571 - mse: 0.2571
Epoch 168/500
63/63 - 0s - loss: 0.2697 - mse: 0.2697
Epoch 169/500
63/63 - 0s - loss: 0.2555 - mse: 0.2555
Epoch 170/500
63/63 - 0s - loss: 0.2794 - mse: 0.2794
Epoch 171/500
63/63 - 0s - loss: 0.2525 - mse: 0.2525
Epoch 172/500
63/63 - 0s - loss: 0.2882 - mse: 0.2882
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.1955 - mse: 0.1955
Epoch 308/500
63/63 - 0s - loss: 0.1939 - mse: 0.1939
Epoch 309/500
63/63 - 0s - loss: 0.1939 - mse: 0.1939
Epoch 310/500
63/63 - 0s - loss: 0.1966 - mse: 0.1966
Epoch 311/500
63/63 - 0s - loss: 0.2004 - mse: 0.2004
Epoch 312/500
63/63 - 0s - loss: 0.2098 - mse: 0.2098
Epoch 313/500
63/63 - 0s - loss: 0.2010 - mse: 0.2010
Epoch 314/500
63/63 - 0s - loss: 0.1919 - mse: 0.1919
Epoch 315/500
63/63 - 0s - loss: 0.2287 - mse: 0.2287
Epoch 316/500
63/63 - 0s - loss: 0.2106 - mse: 0.2106
Epoch 317/500
63/63 - 0s - loss: 0.2206 - mse: 0.2206
Epoch 318/500
63/63 - 0s - loss: 0.1934 - mse: 0.1934
Epoch 319/500
63/63 - 0s - loss: 0.2005 - mse: 0.2005
Epoch 320/500
63/63 - 0s - loss: 0.1869 - mse: 0.1869
Epoch 321/500
63/63 - 0s - loss: 0.1877 - mse: 0.1877
Epoch 322/500
63/63 - 0s - loss: 0.1895 - mse: 0.1895
Epoch 323/500
63/63 - 0s - loss: 0.1845 - mse: 0.1845
Epoch 324/500
63/63 - 0s - loss: 0.1778 - mse: 0.1778
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1753 - mse: 0.1753
Epoch 460/500
63/63 - 0s - loss: 0.1755 - mse: 0.1755
Epoch 461/500
63/63 - 0s - loss: 0.1783 - mse: 0.1783
Epoch 462/500
63/63 - 0s - loss: 0.1546 - mse: 0.1546
Epoch 463/500
63/63 - 0s - loss: 0.1683 - mse: 0.1683
Epoch 464/500
63/63 - 0s - loss: 0.1654 - mse: 0.1654
Epoch 465/500
63/63 - 0s - loss: 0.1537 - mse: 0.1537
Epoch 466/500
63/63 - 0s - loss: 0.1655 - mse: 0.1655
Epoch 467/500
63/63 - 0s - loss: 0.1573 - mse: 0.1573
Epoch 468/500
63/63 - 0s - loss: 0.1602 - mse: 0.1602
Epoch 469/500
63/63 - 0s - loss: 0.1529 - mse: 0.1529
Epoch 470/500
63/63 - 0s - loss: 0.1592 - mse: 0.1592
Epoch 471/500
63/63 - 0s - loss: 0.1583 - mse: 0.1583
Epoch 472/500
63/63 - 0s - loss: 0.1626 - mse: 0.1626
Epoch 473/500
63/63 - 0s - loss: 0.1775 - mse: 0.1775
Epoch 474/500
63/63 - 0s - loss: 0.1890 - mse: 0.1890
Epoch 475/500
63/63 - 0s - loss: 0.1529 - mse: 0.1529
Epoch 476/500
63/63 - 0s - loss: 0.1659 - mse: 0.1659
Epoch 477/500
63/63 - 0s - l

### 5.3 모델 예측 및 평가

In [65]:
pred = MLP_model.predict(stage1['test_X'])

In [66]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.290287


In [67]:
pred = MLP_model_ln.predict(stage1['test_X'])

In [68]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.271939


정규화를 사용한 모델에서 성능이 더 좋습니다.

### [TODO] Stage2에 대해 정규화를 적용해봅니다.

In [69]:
# MLP 모델을 설정합니다.  정규화를 사용할 모델과 사용하지 않을 모델을 모두 정의합니다.


In [70]:
# 모델 컴파일


In [71]:
# 정규화를 사용하지 않을 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 0.8239 - mse: 0.8239
Epoch 2/500
63/63 - 0s - loss: 0.5774 - mse: 0.5774
Epoch 3/500
63/63 - 0s - loss: 0.4982 - mse: 0.4982
Epoch 4/500
63/63 - 0s - loss: 0.4737 - mse: 0.4737
Epoch 5/500
63/63 - 0s - loss: 0.4593 - mse: 0.4593
Epoch 6/500
63/63 - 0s - loss: 0.4453 - mse: 0.4453
Epoch 7/500
63/63 - 0s - loss: 0.4332 - mse: 0.4332
Epoch 8/500
63/63 - 0s - loss: 0.4228 - mse: 0.4228
Epoch 9/500
63/63 - 0s - loss: 0.4129 - mse: 0.4129
Epoch 10/500
63/63 - 0s - loss: 0.4017 - mse: 0.4017
Epoch 11/500
63/63 - 0s - loss: 0.3931 - mse: 0.3931
Epoch 12/500
63/63 - 0s - loss: 0.3989 - mse: 0.3989
Epoch 13/500
63/63 - 0s - loss: 0.3882 - mse: 0.3882
Epoch 14/500
63/63 - 0s - loss: 0.3702 - mse: 0.3702
Epoch 15/500
63/63 - 0s - loss: 0.3602 - mse: 0.3602
Epoch 16/500
63/63 - 0s - loss: 0.3553 - mse: 0.3553
Epoch 17/500
63/63 - 0s - loss: 0.3497 - mse: 0.3497
Epoch 18/500
63/63 - 0s - loss: 0.3394 - mse: 0.3394
Epoch 19/500
63/63 - 0s - loss: 0.3360 - mse: 0.3360
Ep

Epoch 155/500
63/63 - 0s - loss: 0.1343 - mse: 0.1343
Epoch 156/500
63/63 - 0s - loss: 0.1340 - mse: 0.1340
Epoch 157/500
63/63 - 0s - loss: 0.1300 - mse: 0.1300
Epoch 158/500
63/63 - 0s - loss: 0.1278 - mse: 0.1278
Epoch 159/500
63/63 - 0s - loss: 0.1290 - mse: 0.1290
Epoch 160/500
63/63 - 0s - loss: 0.1323 - mse: 0.1323
Epoch 161/500
63/63 - 0s - loss: 0.1272 - mse: 0.1272
Epoch 162/500
63/63 - 0s - loss: 0.1247 - mse: 0.1247
Epoch 163/500
63/63 - 0s - loss: 0.1314 - mse: 0.1314
Epoch 164/500
63/63 - 0s - loss: 0.1289 - mse: 0.1289
Epoch 165/500
63/63 - 0s - loss: 0.1280 - mse: 0.1280
Epoch 166/500
63/63 - 0s - loss: 0.1237 - mse: 0.1237
Epoch 167/500
63/63 - 0s - loss: 0.1250 - mse: 0.1250
Epoch 168/500
63/63 - 0s - loss: 0.1288 - mse: 0.1288
Epoch 169/500
63/63 - 0s - loss: 0.1256 - mse: 0.1256
Epoch 170/500
63/63 - 0s - loss: 0.1286 - mse: 0.1286
Epoch 171/500
63/63 - 0s - loss: 0.1246 - mse: 0.1246
Epoch 172/500
63/63 - 0s - loss: 0.1244 - mse: 0.1244
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.0839 - mse: 0.0839
Epoch 308/500
63/63 - 0s - loss: 0.0775 - mse: 0.0775
Epoch 309/500
63/63 - 0s - loss: 0.0908 - mse: 0.0908
Epoch 310/500
63/63 - 0s - loss: 0.0795 - mse: 0.0795
Epoch 311/500
63/63 - 0s - loss: 0.0768 - mse: 0.0768
Epoch 312/500
63/63 - 0s - loss: 0.0743 - mse: 0.0743
Epoch 313/500
63/63 - 0s - loss: 0.0765 - mse: 0.0765
Epoch 314/500
63/63 - 0s - loss: 0.0804 - mse: 0.0804
Epoch 315/500
63/63 - 0s - loss: 0.0765 - mse: 0.0765
Epoch 316/500
63/63 - 0s - loss: 0.0790 - mse: 0.0790
Epoch 317/500
63/63 - 0s - loss: 0.0811 - mse: 0.0811
Epoch 318/500
63/63 - 0s - loss: 0.0825 - mse: 0.0825
Epoch 319/500
63/63 - 0s - loss: 0.0749 - mse: 0.0749
Epoch 320/500
63/63 - 0s - loss: 0.0770 - mse: 0.0770
Epoch 321/500
63/63 - 0s - loss: 0.0759 - mse: 0.0759
Epoch 322/500
63/63 - 0s - loss: 0.0758 - mse: 0.0758
Epoch 323/500
63/63 - 0s - loss: 0.0736 - mse: 0.0736
Epoch 324/500
63/63 - 0s - loss: 0.0716 - mse: 0.0716
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.0577 - mse: 0.0577
Epoch 460/500
63/63 - 0s - loss: 0.0596 - mse: 0.0596
Epoch 461/500
63/63 - 0s - loss: 0.0579 - mse: 0.0579
Epoch 462/500
63/63 - 0s - loss: 0.0822 - mse: 0.0822
Epoch 463/500
63/63 - 0s - loss: 0.0800 - mse: 0.0800
Epoch 464/500
63/63 - 0s - loss: 0.0762 - mse: 0.0762
Epoch 465/500
63/63 - 0s - loss: 0.0736 - mse: 0.0736
Epoch 466/500
63/63 - 0s - loss: 0.0655 - mse: 0.0655
Epoch 467/500
63/63 - 0s - loss: 0.0626 - mse: 0.0626
Epoch 468/500
63/63 - 0s - loss: 0.0570 - mse: 0.0570
Epoch 469/500
63/63 - 0s - loss: 0.0549 - mse: 0.0549
Epoch 470/500
63/63 - 0s - loss: 0.0530 - mse: 0.0530
Epoch 471/500
63/63 - 0s - loss: 0.0533 - mse: 0.0533
Epoch 472/500
63/63 - 0s - loss: 0.0541 - mse: 0.0541
Epoch 473/500
63/63 - 0s - loss: 0.0522 - mse: 0.0522
Epoch 474/500
63/63 - 0s - loss: 0.0552 - mse: 0.0552
Epoch 475/500
63/63 - 0s - loss: 0.0548 - mse: 0.0548
Epoch 476/500
63/63 - 0s - loss: 0.0541 - mse: 0.0541
Epoch 477/500
63/63 - 0s - l

In [72]:
# 정규화를 사용할 모델을 학습합니다.


Epoch 1/500
63/63 - 1s - loss: 1.1923 - mse: 1.1923
Epoch 2/500
63/63 - 0s - loss: 0.7891 - mse: 0.7891
Epoch 3/500
63/63 - 0s - loss: 0.6841 - mse: 0.6841
Epoch 4/500
63/63 - 0s - loss: 0.6344 - mse: 0.6344
Epoch 5/500
63/63 - 0s - loss: 0.5883 - mse: 0.5883
Epoch 6/500
63/63 - 0s - loss: 0.6011 - mse: 0.6011
Epoch 7/500
63/63 - 0s - loss: 0.5628 - mse: 0.5628
Epoch 8/500
63/63 - 0s - loss: 0.5289 - mse: 0.5289
Epoch 9/500
63/63 - 0s - loss: 0.5198 - mse: 0.5198
Epoch 10/500
63/63 - 0s - loss: 0.5073 - mse: 0.5073
Epoch 11/500
63/63 - 0s - loss: 0.5195 - mse: 0.5195
Epoch 12/500
63/63 - 0s - loss: 0.4988 - mse: 0.4988
Epoch 13/500
63/63 - 0s - loss: 0.4945 - mse: 0.4945
Epoch 14/500
63/63 - 0s - loss: 0.4799 - mse: 0.4799
Epoch 15/500
63/63 - 0s - loss: 0.4912 - mse: 0.4912
Epoch 16/500
63/63 - 0s - loss: 0.5117 - mse: 0.5117
Epoch 17/500
63/63 - 0s - loss: 0.4892 - mse: 0.4892
Epoch 18/500
63/63 - 0s - loss: 0.4835 - mse: 0.4835
Epoch 19/500
63/63 - 0s - loss: 0.4639 - mse: 0.4639
Ep

Epoch 155/500
63/63 - 0s - loss: 0.2851 - mse: 0.2851
Epoch 156/500
63/63 - 0s - loss: 0.3006 - mse: 0.3006
Epoch 157/500
63/63 - 0s - loss: 0.3039 - mse: 0.3039
Epoch 158/500
63/63 - 0s - loss: 0.2886 - mse: 0.2886
Epoch 159/500
63/63 - 0s - loss: 0.2956 - mse: 0.2956
Epoch 160/500
63/63 - 0s - loss: 0.2906 - mse: 0.2906
Epoch 161/500
63/63 - 0s - loss: 0.2845 - mse: 0.2845
Epoch 162/500
63/63 - 0s - loss: 0.2966 - mse: 0.2966
Epoch 163/500
63/63 - 0s - loss: 0.3039 - mse: 0.3039
Epoch 164/500
63/63 - 0s - loss: 0.3106 - mse: 0.3106
Epoch 165/500
63/63 - 0s - loss: 0.2839 - mse: 0.2839
Epoch 166/500
63/63 - 0s - loss: 0.3024 - mse: 0.3024
Epoch 167/500
63/63 - 0s - loss: 0.2952 - mse: 0.2952
Epoch 168/500
63/63 - 0s - loss: 0.2986 - mse: 0.2986
Epoch 169/500
63/63 - 0s - loss: 0.2934 - mse: 0.2934
Epoch 170/500
63/63 - 0s - loss: 0.2875 - mse: 0.2875
Epoch 171/500
63/63 - 0s - loss: 0.2939 - mse: 0.2939
Epoch 172/500
63/63 - 0s - loss: 0.2901 - mse: 0.2901
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2586 - mse: 0.2586
Epoch 308/500
63/63 - 0s - loss: 0.2446 - mse: 0.2446
Epoch 309/500
63/63 - 0s - loss: 0.2448 - mse: 0.2448
Epoch 310/500
63/63 - 0s - loss: 0.2600 - mse: 0.2600
Epoch 311/500
63/63 - 0s - loss: 0.2495 - mse: 0.2495
Epoch 312/500
63/63 - 0s - loss: 0.2601 - mse: 0.2601
Epoch 313/500
63/63 - 0s - loss: 0.2580 - mse: 0.2580
Epoch 314/500
63/63 - 0s - loss: 0.2475 - mse: 0.2475
Epoch 315/500
63/63 - 0s - loss: 0.2403 - mse: 0.2403
Epoch 316/500
63/63 - 0s - loss: 0.2617 - mse: 0.2617
Epoch 317/500
63/63 - 0s - loss: 0.2547 - mse: 0.2547
Epoch 318/500
63/63 - 0s - loss: 0.2638 - mse: 0.2638
Epoch 319/500
63/63 - 0s - loss: 0.2574 - mse: 0.2574
Epoch 320/500
63/63 - 0s - loss: 0.2467 - mse: 0.2467
Epoch 321/500
63/63 - 0s - loss: 0.2461 - mse: 0.2461
Epoch 322/500
63/63 - 0s - loss: 0.2465 - mse: 0.2465
Epoch 323/500
63/63 - 0s - loss: 0.2536 - mse: 0.2536
Epoch 324/500
63/63 - 0s - loss: 0.2460 - mse: 0.2460
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2330 - mse: 0.2330
Epoch 460/500
63/63 - 0s - loss: 0.2175 - mse: 0.2175
Epoch 461/500
63/63 - 0s - loss: 0.2228 - mse: 0.2228
Epoch 462/500
63/63 - 0s - loss: 0.2208 - mse: 0.2208
Epoch 463/500
63/63 - 0s - loss: 0.2304 - mse: 0.2304
Epoch 464/500
63/63 - 0s - loss: 0.2307 - mse: 0.2307
Epoch 465/500
63/63 - 0s - loss: 0.2116 - mse: 0.2116
Epoch 466/500
63/63 - 0s - loss: 0.2444 - mse: 0.2444
Epoch 467/500
63/63 - 0s - loss: 0.2184 - mse: 0.2184
Epoch 468/500
63/63 - 0s - loss: 0.2205 - mse: 0.2205
Epoch 469/500
63/63 - 0s - loss: 0.2193 - mse: 0.2193
Epoch 470/500
63/63 - 0s - loss: 0.2185 - mse: 0.2185
Epoch 471/500
63/63 - 0s - loss: 0.2334 - mse: 0.2334
Epoch 472/500
63/63 - 0s - loss: 0.2253 - mse: 0.2253
Epoch 473/500
63/63 - 0s - loss: 0.2329 - mse: 0.2329
Epoch 474/500
63/63 - 0s - loss: 0.2183 - mse: 0.2183
Epoch 475/500
63/63 - 0s - loss: 0.2213 - mse: 0.2213
Epoch 476/500
63/63 - 0s - loss: 0.2140 - mse: 0.2140
Epoch 477/500
63/63 - 0s - l

In [73]:
# 정규화를 사용하지 않은 모델을 평가합니다.


R2 score: 0.138335


In [74]:
# 정규화를 사용한 모델을 평가합니다.


R2 score: 0.488625


## 6. 데이터 증강 기법
---
금속분말 데이터셋을 이용하여 정규화를 수행해보겠습니다.


### 6.1 모델 정의

In [75]:
# MLP 모델을 설정합니다.
# 데이터 증강을 하지 않을 모델입니다. Dropout에서 학습한 모델을 사용하겠습니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

# 데이터 증강 기법을 사용할 모델입니다. 데이터에 임의의 가우시안 노이즈를 추가해보겠습니다.
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)
MLP_model_aug = tf.keras.Sequential([
    tf.keras.layers.GaussianNoise(stddev = 0.1),
    tf.keras.layers.Dense(128, activation = 'relu'), 
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(stage1['train_y'].shape[1])
])

### 6.2 모델 컴파일

In [76]:
MLP_model.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])
MLP_model_aug.compile(loss = 'mse',
              optimizer = tf.keras.optimizers.SGD(),
              metrics = ['mse'])

### 6.3 모델 학습

In [77]:
# 데이터 증강기법을 사용하지 않을 모델을 학습합니다.
history = MLP_model.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.1605 - mse: 1.1605
Epoch 2/500
63/63 - 0s - loss: 1.1044 - mse: 1.1044
Epoch 3/500
63/63 - 0s - loss: 1.0752 - mse: 1.0752
Epoch 4/500
63/63 - 0s - loss: 1.0524 - mse: 1.0524
Epoch 5/500
63/63 - 0s - loss: 1.0320 - mse: 1.0320
Epoch 6/500
63/63 - 0s - loss: 1.0123 - mse: 1.0123
Epoch 7/500
63/63 - 0s - loss: 0.9935 - mse: 0.9935
Epoch 8/500
63/63 - 0s - loss: 0.9763 - mse: 0.9763
Epoch 9/500
63/63 - 0s - loss: 0.9609 - mse: 0.9609
Epoch 10/500
63/63 - 0s - loss: 0.9462 - mse: 0.9462
Epoch 11/500
63/63 - 0s - loss: 0.9324 - mse: 0.9324
Epoch 12/500
63/63 - 0s - loss: 0.9203 - mse: 0.9203
Epoch 13/500
63/63 - 0s - loss: 0.9085 - mse: 0.9085
Epoch 14/500
63/63 - 0s - loss: 0.8980 - mse: 0.8980
Epoch 15/500
63/63 - 0s - loss: 0.8886 - mse: 0.8886
Epoch 16/500
63/63 - 0s - loss: 0.8800 - mse: 0.8800
Epoch 17/500
63/63 - 0s - loss: 0.8723 - mse: 0.8723
Epoch 18/500
63/63 - 0s - loss: 0.8655 - mse: 0.8655
Epoch 19/500
63/63 - 0s - loss: 0.8590 - mse: 0.8590
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3062 - mse: 0.3062
Epoch 156/500
63/63 - 0s - loss: 0.3051 - mse: 0.3051
Epoch 157/500
63/63 - 0s - loss: 0.3044 - mse: 0.3044
Epoch 158/500
63/63 - 0s - loss: 0.3036 - mse: 0.3036
Epoch 159/500
63/63 - 0s - loss: 0.3028 - mse: 0.3028
Epoch 160/500
63/63 - 0s - loss: 0.3022 - mse: 0.3022
Epoch 161/500
63/63 - 0s - loss: 0.3010 - mse: 0.3010
Epoch 162/500
63/63 - 0s - loss: 0.2998 - mse: 0.2998
Epoch 163/500
63/63 - 0s - loss: 0.2994 - mse: 0.2994
Epoch 164/500
63/63 - 0s - loss: 0.2981 - mse: 0.2981
Epoch 165/500
63/63 - 0s - loss: 0.2979 - mse: 0.2979
Epoch 166/500
63/63 - 0s - loss: 0.2973 - mse: 0.2973
Epoch 167/500
63/63 - 0s - loss: 0.2962 - mse: 0.2962
Epoch 168/500
63/63 - 0s - loss: 0.2952 - mse: 0.2952
Epoch 169/500
63/63 - 0s - loss: 0.2946 - mse: 0.2946
Epoch 170/500
63/63 - 0s - loss: 0.2938 - mse: 0.2938
Epoch 171/500
63/63 - 0s - loss: 0.2929 - mse: 0.2929
Epoch 172/500
63/63 - 0s - loss: 0.2923 - mse: 0.2923
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2174 - mse: 0.2174
Epoch 308/500
63/63 - 0s - loss: 0.2168 - mse: 0.2168
Epoch 309/500
63/63 - 0s - loss: 0.2161 - mse: 0.2161
Epoch 310/500
63/63 - 0s - loss: 0.2160 - mse: 0.2160
Epoch 311/500
63/63 - 0s - loss: 0.2150 - mse: 0.2150
Epoch 312/500
63/63 - 0s - loss: 0.2147 - mse: 0.2147
Epoch 313/500
63/63 - 0s - loss: 0.2142 - mse: 0.2142
Epoch 314/500
63/63 - 0s - loss: 0.2140 - mse: 0.2140
Epoch 315/500
63/63 - 0s - loss: 0.2136 - mse: 0.2136
Epoch 316/500
63/63 - 0s - loss: 0.2130 - mse: 0.2130
Epoch 317/500
63/63 - 0s - loss: 0.2125 - mse: 0.2125
Epoch 318/500
63/63 - 0s - loss: 0.2118 - mse: 0.2118
Epoch 319/500
63/63 - 0s - loss: 0.2116 - mse: 0.2116
Epoch 320/500
63/63 - 0s - loss: 0.2108 - mse: 0.2108
Epoch 321/500
63/63 - 0s - loss: 0.2103 - mse: 0.2103
Epoch 322/500
63/63 - 0s - loss: 0.2100 - mse: 0.2100
Epoch 323/500
63/63 - 0s - loss: 0.2092 - mse: 0.2092
Epoch 324/500
63/63 - 0s - loss: 0.2089 - mse: 0.2089
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1534 - mse: 0.1534
Epoch 460/500
63/63 - 0s - loss: 0.1528 - mse: 0.1528
Epoch 461/500
63/63 - 0s - loss: 0.1528 - mse: 0.1528
Epoch 462/500
63/63 - 0s - loss: 0.1527 - mse: 0.1527
Epoch 463/500
63/63 - 0s - loss: 0.1519 - mse: 0.1519
Epoch 464/500
63/63 - 0s - loss: 0.1513 - mse: 0.1513
Epoch 465/500
63/63 - 0s - loss: 0.1512 - mse: 0.1512
Epoch 466/500
63/63 - 0s - loss: 0.1509 - mse: 0.1509
Epoch 467/500
63/63 - 0s - loss: 0.1508 - mse: 0.1508
Epoch 468/500
63/63 - 0s - loss: 0.1501 - mse: 0.1501
Epoch 469/500
63/63 - 0s - loss: 0.1503 - mse: 0.1503
Epoch 470/500
63/63 - 0s - loss: 0.1498 - mse: 0.1498
Epoch 471/500
63/63 - 0s - loss: 0.1493 - mse: 0.1493
Epoch 472/500
63/63 - 0s - loss: 0.1491 - mse: 0.1491
Epoch 473/500
63/63 - 0s - loss: 0.1490 - mse: 0.1490
Epoch 474/500
63/63 - 0s - loss: 0.1482 - mse: 0.1482
Epoch 475/500
63/63 - 0s - loss: 0.1482 - mse: 0.1482
Epoch 476/500
63/63 - 0s - loss: 0.1475 - mse: 0.1475
Epoch 477/500
63/63 - 0s - l

In [78]:
# 데이터 증강기법을 사용할 모델을 학습합니다.
history = MLP_model_aug.fit(stage1['train_X'][:1000], stage1['train_y'][:1000], epochs = 500, batch_size = 16, verbose = 2)

Epoch 1/500
63/63 - 0s - loss: 1.1651 - mse: 1.1651
Epoch 2/500
63/63 - 0s - loss: 1.1023 - mse: 1.1023
Epoch 3/500
63/63 - 0s - loss: 1.0662 - mse: 1.0662
Epoch 4/500
63/63 - 0s - loss: 1.0320 - mse: 1.0320
Epoch 5/500
63/63 - 0s - loss: 1.0003 - mse: 1.0003
Epoch 6/500
63/63 - 0s - loss: 0.9731 - mse: 0.9731
Epoch 7/500
63/63 - 0s - loss: 0.9510 - mse: 0.9510
Epoch 8/500
63/63 - 0s - loss: 0.9349 - mse: 0.9349
Epoch 9/500
63/63 - 0s - loss: 0.9204 - mse: 0.9204
Epoch 10/500
63/63 - 0s - loss: 0.9063 - mse: 0.9063
Epoch 11/500
63/63 - 0s - loss: 0.8967 - mse: 0.8967
Epoch 12/500
63/63 - 0s - loss: 0.8879 - mse: 0.8879
Epoch 13/500
63/63 - 0s - loss: 0.8808 - mse: 0.8808
Epoch 14/500
63/63 - 0s - loss: 0.8730 - mse: 0.8730
Epoch 15/500
63/63 - 0s - loss: 0.8642 - mse: 0.8642
Epoch 16/500
63/63 - 0s - loss: 0.8587 - mse: 0.8587
Epoch 17/500
63/63 - 0s - loss: 0.8524 - mse: 0.8524
Epoch 18/500
63/63 - 0s - loss: 0.8471 - mse: 0.8471
Epoch 19/500
63/63 - 0s - loss: 0.8407 - mse: 0.8407
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3039 - mse: 0.3039
Epoch 156/500
63/63 - 0s - loss: 0.2962 - mse: 0.2962
Epoch 157/500
63/63 - 0s - loss: 0.2973 - mse: 0.2973
Epoch 158/500
63/63 - 0s - loss: 0.3015 - mse: 0.3015
Epoch 159/500
63/63 - 0s - loss: 0.2960 - mse: 0.2960
Epoch 160/500
63/63 - 0s - loss: 0.2975 - mse: 0.2975
Epoch 161/500
63/63 - 0s - loss: 0.2908 - mse: 0.2908
Epoch 162/500
63/63 - 0s - loss: 0.2903 - mse: 0.2903
Epoch 163/500
63/63 - 0s - loss: 0.2968 - mse: 0.2968
Epoch 164/500
63/63 - 0s - loss: 0.2840 - mse: 0.2840
Epoch 165/500
63/63 - 0s - loss: 0.2914 - mse: 0.2914
Epoch 166/500
63/63 - 0s - loss: 0.2939 - mse: 0.2939
Epoch 167/500
63/63 - 0s - loss: 0.2904 - mse: 0.2904
Epoch 168/500
63/63 - 0s - loss: 0.2858 - mse: 0.2858
Epoch 169/500
63/63 - 0s - loss: 0.2818 - mse: 0.2818
Epoch 170/500
63/63 - 0s - loss: 0.2829 - mse: 0.2829
Epoch 171/500
63/63 - 0s - loss: 0.2835 - mse: 0.2835
Epoch 172/500
63/63 - 0s - loss: 0.2829 - mse: 0.2829
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2159 - mse: 0.2159
Epoch 308/500
63/63 - 0s - loss: 0.2083 - mse: 0.2083
Epoch 309/500
63/63 - 0s - loss: 0.2063 - mse: 0.2063
Epoch 310/500
63/63 - 0s - loss: 0.2051 - mse: 0.2051
Epoch 311/500
63/63 - 0s - loss: 0.2111 - mse: 0.2111
Epoch 312/500
63/63 - 0s - loss: 0.2096 - mse: 0.2096
Epoch 313/500
63/63 - 0s - loss: 0.2066 - mse: 0.2066
Epoch 314/500
63/63 - 0s - loss: 0.2084 - mse: 0.2084
Epoch 315/500
63/63 - 0s - loss: 0.2029 - mse: 0.2029
Epoch 316/500
63/63 - 0s - loss: 0.2041 - mse: 0.2041
Epoch 317/500
63/63 - 0s - loss: 0.2053 - mse: 0.2053
Epoch 318/500
63/63 - 0s - loss: 0.2047 - mse: 0.2047
Epoch 319/500
63/63 - 0s - loss: 0.2028 - mse: 0.2028
Epoch 320/500
63/63 - 0s - loss: 0.2055 - mse: 0.2055
Epoch 321/500
63/63 - 0s - loss: 0.2024 - mse: 0.2024
Epoch 322/500
63/63 - 0s - loss: 0.2039 - mse: 0.2039
Epoch 323/500
63/63 - 0s - loss: 0.2015 - mse: 0.2015
Epoch 324/500
63/63 - 0s - loss: 0.2015 - mse: 0.2015
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.1562 - mse: 0.1562
Epoch 460/500
63/63 - 0s - loss: 0.1563 - mse: 0.1563
Epoch 461/500
63/63 - 0s - loss: 0.1546 - mse: 0.1546
Epoch 462/500
63/63 - 0s - loss: 0.1572 - mse: 0.1572
Epoch 463/500
63/63 - 0s - loss: 0.1574 - mse: 0.1574
Epoch 464/500
63/63 - 0s - loss: 0.1536 - mse: 0.1536
Epoch 465/500
63/63 - 0s - loss: 0.1534 - mse: 0.1534
Epoch 466/500
63/63 - 0s - loss: 0.1546 - mse: 0.1546
Epoch 467/500
63/63 - 0s - loss: 0.1538 - mse: 0.1538
Epoch 468/500
63/63 - 0s - loss: 0.1556 - mse: 0.1556
Epoch 469/500
63/63 - 0s - loss: 0.1530 - mse: 0.1530
Epoch 470/500
63/63 - 0s - loss: 0.1530 - mse: 0.1530
Epoch 471/500
63/63 - 0s - loss: 0.1506 - mse: 0.1506
Epoch 472/500
63/63 - 0s - loss: 0.1541 - mse: 0.1541
Epoch 473/500
63/63 - 0s - loss: 0.1570 - mse: 0.1570
Epoch 474/500
63/63 - 0s - loss: 0.1498 - mse: 0.1498
Epoch 475/500
63/63 - 0s - loss: 0.1512 - mse: 0.1512
Epoch 476/500
63/63 - 0s - loss: 0.1479 - mse: 0.1479
Epoch 477/500
63/63 - 0s - l

### 6.4 모델 예측 및 평가

In [79]:
pred = MLP_model.predict(stage1['test_X'])

In [80]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.298804


In [81]:
pred = MLP_model_aug.predict(stage1['test_X'])

In [82]:
r2 = sklearn.metrics.r2_score(stage1['test_y'], pred)
print("R2 score: %f"%r2)

R2 score: 0.337222


데이터 증강 기법을 사용한 모델이 성능이 더 좋은 것을 확인할 수 있습니다.

### [TODO] Stage2에 대해 데이터 증강기법을 적용해봅니다.

In [83]:
# MLP 모델을 설정합니다. 데이터증강기법을 사용하지 않을 모델과 사용할 모델을 모두 정의합니다.


In [84]:
# 모델 컴파일


In [85]:
# 데이터 증강기법을 사용하지 않을 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0007 - mse: 1.0007
Epoch 2/500
63/63 - 0s - loss: 0.8786 - mse: 0.8786
Epoch 3/500
63/63 - 0s - loss: 0.7753 - mse: 0.7753
Epoch 4/500
63/63 - 0s - loss: 0.6962 - mse: 0.6962
Epoch 5/500
63/63 - 0s - loss: 0.6453 - mse: 0.6453
Epoch 6/500
63/63 - 0s - loss: 0.6155 - mse: 0.6155
Epoch 7/500
63/63 - 0s - loss: 0.5943 - mse: 0.5943
Epoch 8/500
63/63 - 0s - loss: 0.5782 - mse: 0.5782
Epoch 9/500
63/63 - 0s - loss: 0.5644 - mse: 0.5644
Epoch 10/500
63/63 - 0s - loss: 0.5515 - mse: 0.5515
Epoch 11/500
63/63 - 0s - loss: 0.5415 - mse: 0.5415
Epoch 12/500
63/63 - 0s - loss: 0.5322 - mse: 0.5322
Epoch 13/500
63/63 - 0s - loss: 0.5234 - mse: 0.5234
Epoch 14/500
63/63 - 0s - loss: 0.5148 - mse: 0.5148
Epoch 15/500
63/63 - 0s - loss: 0.5086 - mse: 0.5086
Epoch 16/500
63/63 - 0s - loss: 0.5013 - mse: 0.5013
Epoch 17/500
63/63 - 0s - loss: 0.4955 - mse: 0.4955
Epoch 18/500
63/63 - 0s - loss: 0.4904 - mse: 0.4904
Epoch 19/500
63/63 - 0s - loss: 0.4858 - mse: 0.4858
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3337 - mse: 0.3337
Epoch 156/500
63/63 - 0s - loss: 0.3329 - mse: 0.3329
Epoch 157/500
63/63 - 0s - loss: 0.3331 - mse: 0.3331
Epoch 158/500
63/63 - 0s - loss: 0.3326 - mse: 0.3326
Epoch 159/500
63/63 - 0s - loss: 0.3315 - mse: 0.3315
Epoch 160/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 161/500
63/63 - 0s - loss: 0.3302 - mse: 0.3302
Epoch 162/500
63/63 - 0s - loss: 0.3300 - mse: 0.3300
Epoch 163/500
63/63 - 0s - loss: 0.3292 - mse: 0.3292
Epoch 164/500
63/63 - 0s - loss: 0.3286 - mse: 0.3286
Epoch 165/500
63/63 - 0s - loss: 0.3283 - mse: 0.3283
Epoch 166/500
63/63 - 0s - loss: 0.3275 - mse: 0.3275
Epoch 167/500
63/63 - 0s - loss: 0.3277 - mse: 0.3277
Epoch 168/500
63/63 - 0s - loss: 0.3265 - mse: 0.3265
Epoch 169/500
63/63 - 0s - loss: 0.3264 - mse: 0.3264
Epoch 170/500
63/63 - 0s - loss: 0.3250 - mse: 0.3250
Epoch 171/500
63/63 - 0s - loss: 0.3263 - mse: 0.3263
Epoch 172/500
63/63 - 0s - loss: 0.3232 - mse: 0.3232
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2813 - mse: 0.2813
Epoch 308/500
63/63 - 0s - loss: 0.2808 - mse: 0.2808
Epoch 309/500
63/63 - 0s - loss: 0.2813 - mse: 0.2813
Epoch 310/500
63/63 - 0s - loss: 0.2803 - mse: 0.2803
Epoch 311/500
63/63 - 0s - loss: 0.2799 - mse: 0.2799
Epoch 312/500
63/63 - 0s - loss: 0.2794 - mse: 0.2794
Epoch 313/500
63/63 - 0s - loss: 0.2798 - mse: 0.2798
Epoch 314/500
63/63 - 0s - loss: 0.2797 - mse: 0.2797
Epoch 315/500
63/63 - 0s - loss: 0.2787 - mse: 0.2787
Epoch 316/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 317/500
63/63 - 0s - loss: 0.2784 - mse: 0.2784
Epoch 318/500
63/63 - 0s - loss: 0.2774 - mse: 0.2774
Epoch 319/500
63/63 - 0s - loss: 0.2788 - mse: 0.2788
Epoch 320/500
63/63 - 0s - loss: 0.2780 - mse: 0.2780
Epoch 321/500
63/63 - 0s - loss: 0.2777 - mse: 0.2777
Epoch 322/500
63/63 - 0s - loss: 0.2785 - mse: 0.2785
Epoch 323/500
63/63 - 0s - loss: 0.2781 - mse: 0.2781
Epoch 324/500
63/63 - 0s - loss: 0.2778 - mse: 0.2778
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2526 - mse: 0.2526
Epoch 460/500
63/63 - 0s - loss: 0.2515 - mse: 0.2515
Epoch 461/500
63/63 - 0s - loss: 0.2519 - mse: 0.2519
Epoch 462/500
63/63 - 0s - loss: 0.2533 - mse: 0.2533
Epoch 463/500
63/63 - 0s - loss: 0.2524 - mse: 0.2524
Epoch 464/500
63/63 - 0s - loss: 0.2512 - mse: 0.2512
Epoch 465/500
63/63 - 0s - loss: 0.2515 - mse: 0.2515
Epoch 466/500
63/63 - 0s - loss: 0.2517 - mse: 0.2517
Epoch 467/500
63/63 - 0s - loss: 0.2505 - mse: 0.2505
Epoch 468/500
63/63 - 0s - loss: 0.2508 - mse: 0.2508
Epoch 469/500
63/63 - 0s - loss: 0.2502 - mse: 0.2502
Epoch 470/500
63/63 - 0s - loss: 0.2512 - mse: 0.2512
Epoch 471/500
63/63 - 0s - loss: 0.2506 - mse: 0.2506
Epoch 472/500
63/63 - 0s - loss: 0.2501 - mse: 0.2501
Epoch 473/500
63/63 - 0s - loss: 0.2500 - mse: 0.2500
Epoch 474/500
63/63 - 0s - loss: 0.2497 - mse: 0.2497
Epoch 475/500
63/63 - 0s - loss: 0.2496 - mse: 0.2496
Epoch 476/500
63/63 - 0s - loss: 0.2497 - mse: 0.2497
Epoch 477/500
63/63 - 0s - l

In [86]:
# 데이터 증강기법을 사용할 모델을 학습합니다.


Epoch 1/500
63/63 - 0s - loss: 1.0055 - mse: 1.0055
Epoch 2/500
63/63 - 0s - loss: 0.8742 - mse: 0.8742
Epoch 3/500
63/63 - 0s - loss: 0.7787 - mse: 0.7787
Epoch 4/500
63/63 - 0s - loss: 0.7073 - mse: 0.7073
Epoch 5/500
63/63 - 0s - loss: 0.6627 - mse: 0.6627
Epoch 6/500
63/63 - 0s - loss: 0.6358 - mse: 0.6358
Epoch 7/500
63/63 - 0s - loss: 0.6185 - mse: 0.6185
Epoch 8/500
63/63 - 0s - loss: 0.6037 - mse: 0.6037
Epoch 9/500
63/63 - 0s - loss: 0.5918 - mse: 0.5918
Epoch 10/500
63/63 - 0s - loss: 0.5815 - mse: 0.5815
Epoch 11/500
63/63 - 0s - loss: 0.5711 - mse: 0.5711
Epoch 12/500
63/63 - 0s - loss: 0.5626 - mse: 0.5626
Epoch 13/500
63/63 - 0s - loss: 0.5557 - mse: 0.5557
Epoch 14/500
63/63 - 0s - loss: 0.5476 - mse: 0.5476
Epoch 15/500
63/63 - 0s - loss: 0.5413 - mse: 0.5413
Epoch 16/500
63/63 - 0s - loss: 0.5324 - mse: 0.5324
Epoch 17/500
63/63 - 0s - loss: 0.5262 - mse: 0.5262
Epoch 18/500
63/63 - 0s - loss: 0.5213 - mse: 0.5213
Epoch 19/500
63/63 - 0s - loss: 0.5141 - mse: 0.5141
Ep

Epoch 155/500
63/63 - 0s - loss: 0.3501 - mse: 0.3501
Epoch 156/500
63/63 - 0s - loss: 0.3479 - mse: 0.3479
Epoch 157/500
63/63 - 0s - loss: 0.3506 - mse: 0.3506
Epoch 158/500
63/63 - 0s - loss: 0.3496 - mse: 0.3496
Epoch 159/500
63/63 - 0s - loss: 0.3486 - mse: 0.3486
Epoch 160/500
63/63 - 0s - loss: 0.3477 - mse: 0.3477
Epoch 161/500
63/63 - 0s - loss: 0.3459 - mse: 0.3459
Epoch 162/500
63/63 - 0s - loss: 0.3441 - mse: 0.3441
Epoch 163/500
63/63 - 0s - loss: 0.3440 - mse: 0.3440
Epoch 164/500
63/63 - 0s - loss: 0.3475 - mse: 0.3475
Epoch 165/500
63/63 - 0s - loss: 0.3453 - mse: 0.3453
Epoch 166/500
63/63 - 0s - loss: 0.3468 - mse: 0.3468
Epoch 167/500
63/63 - 0s - loss: 0.3459 - mse: 0.3459
Epoch 168/500
63/63 - 0s - loss: 0.3429 - mse: 0.3429
Epoch 169/500
63/63 - 0s - loss: 0.3447 - mse: 0.3447
Epoch 170/500
63/63 - 0s - loss: 0.3409 - mse: 0.3409
Epoch 171/500
63/63 - 0s - loss: 0.3424 - mse: 0.3424
Epoch 172/500
63/63 - 0s - loss: 0.3391 - mse: 0.3391
Epoch 173/500
63/63 - 0s - l

Epoch 307/500
63/63 - 0s - loss: 0.2978 - mse: 0.2978
Epoch 308/500
63/63 - 0s - loss: 0.2980 - mse: 0.2980
Epoch 309/500
63/63 - 0s - loss: 0.2981 - mse: 0.2981
Epoch 310/500
63/63 - 0s - loss: 0.3004 - mse: 0.3004
Epoch 311/500
63/63 - 0s - loss: 0.2970 - mse: 0.2970
Epoch 312/500
63/63 - 0s - loss: 0.2931 - mse: 0.2931
Epoch 313/500
63/63 - 0s - loss: 0.2944 - mse: 0.2944
Epoch 314/500
63/63 - 0s - loss: 0.2961 - mse: 0.2961
Epoch 315/500
63/63 - 0s - loss: 0.2918 - mse: 0.2918
Epoch 316/500
63/63 - 0s - loss: 0.2989 - mse: 0.2989
Epoch 317/500
63/63 - 0s - loss: 0.2935 - mse: 0.2935
Epoch 318/500
63/63 - 0s - loss: 0.2951 - mse: 0.2951
Epoch 319/500
63/63 - 0s - loss: 0.2983 - mse: 0.2983
Epoch 320/500
63/63 - 0s - loss: 0.2991 - mse: 0.2991
Epoch 321/500
63/63 - 0s - loss: 0.2939 - mse: 0.2939
Epoch 322/500
63/63 - 0s - loss: 0.2949 - mse: 0.2949
Epoch 323/500
63/63 - 0s - loss: 0.2974 - mse: 0.2974
Epoch 324/500
63/63 - 0s - loss: 0.2955 - mse: 0.2955
Epoch 325/500
63/63 - 0s - l

Epoch 459/500
63/63 - 0s - loss: 0.2691 - mse: 0.2691
Epoch 460/500
63/63 - 0s - loss: 0.2675 - mse: 0.2675
Epoch 461/500
63/63 - 0s - loss: 0.2650 - mse: 0.2650
Epoch 462/500
63/63 - 0s - loss: 0.2665 - mse: 0.2665
Epoch 463/500
63/63 - 0s - loss: 0.2711 - mse: 0.2711
Epoch 464/500
63/63 - 0s - loss: 0.2708 - mse: 0.2708
Epoch 465/500
63/63 - 0s - loss: 0.2684 - mse: 0.2684
Epoch 466/500
63/63 - 0s - loss: 0.2674 - mse: 0.2674
Epoch 467/500
63/63 - 0s - loss: 0.2704 - mse: 0.2704
Epoch 468/500
63/63 - 0s - loss: 0.2663 - mse: 0.2663
Epoch 469/500
63/63 - 0s - loss: 0.2664 - mse: 0.2664
Epoch 470/500
63/63 - 0s - loss: 0.2664 - mse: 0.2664
Epoch 471/500
63/63 - 0s - loss: 0.2651 - mse: 0.2651
Epoch 472/500
63/63 - 0s - loss: 0.2645 - mse: 0.2645
Epoch 473/500
63/63 - 0s - loss: 0.2649 - mse: 0.2649
Epoch 474/500
63/63 - 0s - loss: 0.2655 - mse: 0.2655
Epoch 475/500
63/63 - 0s - loss: 0.2664 - mse: 0.2664
Epoch 476/500
63/63 - 0s - loss: 0.2689 - mse: 0.2689
Epoch 477/500
63/63 - 0s - l

In [87]:
# 정규화를 사용하지 않은 모델을 평가합니다.


R2 score: 0.586051


In [88]:
# 정규화를 사용한 모델을 평가합니다.


R2 score: 0.447476


<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>